# IMPORT THE LIBRARIES

In [19]:
#exc1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from dateutil.parser import parse
from itertools import compress
from collections import OrderedDict
from datetime import date
import datetime
from datetime import timedelta
import re

# LOAD THE PI DATA

In [2]:
#Sheet From Pi
#Step1: Load the Data for the month
df_JAN2019Pi = pd.read_excel(r'C:\Users\v.t.flores\Documents\Pressure_Filter_2020_PiData.xlsx', sheet_name='FromPi_JAN2020', 
                             index_col=False)

#Drop the row[1]
df_JAN2019Pi = df_JAN2019Pi.drop(df_JAN2019Pi.index[0])

#Step2: Parse the DATETIME column
df_JAN2019Pi['DATETIME'] = df_JAN2019Pi['DATETIME'].apply(lambda x: parse(str(x)))

df_JAN2019Pi.set_index('DATETIME', inplace=True)
df_comb = df_JAN2019Pi
df_comb = df_comb.dropna()

In [3]:
df_comb.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 43200 entries, 2020-01-01 00:00:00 to 2020-01-30 23:59:00
Data columns (total 28 columns):
A_t_CWSH1         43200 non-null object
B_t_CWSH1         43200 non-null object
A_t_CWSH2         43200 non-null object
B_t_CWSH2         43200 non-null object
A_t_DRY           43200 non-null object
B_t_DRY           43200 non-null object
A_t_FEED          43200 non-null object
B_t_FEED          43200 non-null object
A_t_PRESS1        43200 non-null object
A_t_PRESS2        43200 non-null object
B_t_PRESS1        43200 non-null object
B_t_PRESS2        43200 non-null object
106PU26A_CUR      43200 non-null object
PU26A_OP          43200 non-null object
PU27A_OP          43200 non-null object
106PU27A_CUR      43200 non-null object
106FT02A_MANFP    43200 non-null object
106FT02A_CWP      43200 non-null object
106FT02A_TOTWT    43200 non-null object
106PU26B_CUR      43200 non-null object
106PU27B_CUR      43200 non-null object
PU26B_OP        

# SELECT THE ALPHA PARAMETERS

In [4]:
#exc7
#finding peaks FT for A_filtration time

df_comb['A_t_FEED'] = pd.to_numeric(df_comb['A_t_FEED'], errors='coerce')
a = np.diff(np.sign(np.diff(df_comb['A_t_FEED']))).nonzero()[0] + 1               # local min & max
b = (np.diff(np.sign(np.diff(df_comb['A_t_FEED']))) > 0).nonzero()[0] + 1         # local min
c = (np.diff(np.sign(np.diff(df_comb['A_t_FEED']))) < 0).nonzero()[0] + 1         # local max
# +1 due to the fact that diff reduces the original index number


#finding peaks FT for A_N2 Dry time

df_comb['A_t_DRY'] = pd.to_numeric(df_comb['A_t_DRY'], errors='coerce')
d = np.diff(np.sign(np.diff(df_comb['A_t_DRY']))).nonzero()[0] + 1               # local min & max
e = (np.diff(np.sign(np.diff(df_comb['A_t_DRY']))) > 0).nonzero()[0] + 1         # local min
f = (np.diff(np.sign(np.diff(df_comb['A_t_DRY']))) < 0).nonzero()[0] + 1         # local max
# +1 due to the fact that diff reduces the original index number

#Use b as the starting index and f as the ending index for one full cylcle of Filtration to N2 Drying

#-----------------------------------------------------------------------------------------------------------------------

#This code here collects the index range for the each cycle
try:
    ft_cyc_lim = {}
    for i in range(0, len(b)):
        for j in range(0, len(f)):
            if f[j] > b[i] and f[j] < b[i+1]:
                if f[j+1]<b[i+1]:
                    cyc_start = b[i]
                    cyc_end = f[j+1]
                    ft_cyc_lim.setdefault(cyc_start,cyc_end)
except IndexError:
    print('\nDone')
#------------------------------------------------------------  
print('Done', str(datetime.datetime.now()))
print(cyc_start, cyc_end)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in sign
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in sign
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in greater
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in sign
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in less
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in sign
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in sign
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: in


Done
Done 2020-07-13 09:54:56.623466
37930 37954


In [5]:
keyslist = list(ft_cyc_lim.keys()) #creates a list of the ft_cyc_lim keys so that we can iterate later

#Remove the 'No Good Data For Calculation' values using for loop
for x in range(0, len(df_comb['106FT02A_CWP'])):
    if df_comb['106FT02A_CWP'][x] =='[-11059] No Good Data For Calculation':
            df_comb['106FT02A_CWP'][x] = 0


#code for getting the CWP for CakeWash1 and CakeWash2

A_CWP_1new = list()
A_CWP_1new[:] = []

A_CWP_1_indexList = list()
A_CWP_1_indexList[:] = []

A_CWP_2new = list()
A_CWP_2new[:] = []

A_CWP_2_indexList = list()
A_CWP_2_indexList[:] = []

A_CWP_1_inner = list()
A_CWP_1_inner[:] = []

A_CWP_2_inner = list()
A_CWP_2_inner[:] = []

A_Pair_CWP_1_List = list()
A_Pair_CWP_2_List = list()

try:
    for i in range(0, len(keyslist)):
        A_CWPList_i = list()
        A_CWPList_i[:] = []
        d = list()
        d[:] = []
        
        A_CWPList_i = list(np.arange(keyslist[i],ft_cyc_lim[keyslist[i]]))
        d = (np.diff(np.sign(np.diff(df_comb['106FT02A_CWP'].iloc[keyslist[i]:ft_cyc_lim[keyslist[i]]])))< 0).nonzero()[0] + 1
        A_CWP_for_sort = list()
        A_CWP_for_sort[:] = [] 
        
        A_CWP_1_indexList.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))
        A_CWP_2_indexList.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))
        
        for i in range(0, len(d)):
            CWP_val_shortlist = df_comb['106FT02A_CWP'].iloc[A_CWPList_i[d[i]]]
            A_CWP_for_sort.append(CWP_val_shortlist)
                    
        A_CWP_1 = sorted(A_CWP_for_sort)[len(A_CWP_for_sort)-1]
        A_CWP_1_inner.append(A_CWP_1)
        
        A_CWP_2 = sorted(A_CWP_for_sort)[len(A_CWP_for_sort)-2]
        A_CWP_2_inner.append(A_CWP_2)

    A_Pair_CWP_1_List = list(zip(A_CWP_1_inner, A_CWP_1_indexList[0:]))
    A_Pair_CWP_2_List = list(zip(A_CWP_2_inner, A_CWP_2_indexList[0:]))
            
    
except IndexError:
    print('IndexError')

df_A_Pair_CWP_1 = pd.DataFrame(A_Pair_CWP_1_List)
df_A_Pair_CWP_1.columns = ['A_CWP_1', 'DATETIME']
df_A_Pair_CWP_1.set_index('DATETIME', inplace=True)
    
df_A_Pair_CWP_2 = pd.DataFrame(A_Pair_CWP_2_List)
df_A_Pair_CWP_2.columns = ['A_CWP_2', 'DATETIME']
df_A_Pair_CWP_2.set_index('DATETIME', inplace=True)

print('Done', str(datetime.datetime.now()))


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Done 2020-07-13 09:54:57.504408


In [6]:
# This code will use the index range (or limits) to create a list of values: feedtime range, cake wash time range,
# then selects the max value as the parameter value for that range


#Remove the 'No Good Data For Calculation' values using for loop
for x in range(0, len(df_comb['A_t_FEED'])):
    if df_comb['A_t_FEED'][x] =='[-11059] No Good Data For Calculation':
            df_comb['A_t_FEED'][x] = 0

#FILTRATION TIME
keyslist = list(ft_cyc_lim.keys()) #creates a list of the ft_cyc_lim keys so that we can iterate later
#For loop Dictionary names 
A_Filt_MaxList= list()
A_Filt_MaxList[:] = []
A_feedt_Val_List = list()
A_feedt_Val_List[:] = []
A_feedt_index_List = list()
A_feedt_index_List[:] = []
A_Pair_Filt_List = list()
A_Pair_Filt_List[:] = []
try:         
    for i in range(0, len(keyslist)):
        A_feedt_Val_List = list()
        A_feedt_Val_List[:] = []
        for j in range(keyslist[i], ft_cyc_lim[keyslist[i]]):
            A_feedt = df_comb['A_t_FEED'][j]
            A_feedt_Val_List.append(A_feedt)
        A_Filt_MaxList.append(sorted(A_feedt_Val_List)[-1])
        A_feedt_index_List.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))  
    A_Pair_Filt_List = list(zip(A_Filt_MaxList, A_feedt_index_List[0:]))      
except:
    pass

df_A_Pair_Filt_t = pd.DataFrame(A_Pair_Filt_List)
df_A_Pair_Filt_t.columns = ['A_FILT_T', 'DATETIME']
df_A_Pair_Filt_t.set_index('DATETIME', inplace=True)

print('Done', str(datetime.datetime.now()))
df_comb['A_t_FEED'].max()

Done 2020-07-13 09:54:59.287583


1000.0

In [7]:
#MANIFOLD PRESSURE

#Remove the 'No Good Data For Calculation' values using for loop
for x in range(0, len(df_comb['106FT02A_MANFP'])):
    if df_comb['106FT02A_MANFP'][x] =='[-11059] No Good Data For Calculation':
            df_comb['106FT02A_MANFP'][x] = 0

keyslist = list(ft_cyc_lim.keys()) #creates a list of the ft_cyc_lim keys so that we can iterate later
#For loop Dictionary names 
A_MANFP_MaxList = list()
A_MANFP_MaxList[:] = []
A_Pair_MANFP_List =list() 
A_Pair_MANFP_List[:] = []
A_MANFP_index_List=list()      
A_MANFP_index_List[:] = []
try:         
    for i in range(0, len(keyslist)):
        A_MANFP_val_List=list() 
        A_MANFP_val_List[:] = []
        for j in range(keyslist[i], ft_cyc_lim[keyslist[i]]):               
            #Gets A manifold pressure of the cycle
            A_MANFP = df_comb['106FT02A_MANFP'][j]         
            A_MANFP_val_List.append(A_MANFP)
        A_MANFP_MaxList.append(sorted(A_MANFP_val_List)[-1])
        A_MANFP_index_List.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))
    A_Pair_MANFP_List = list(zip(A_MANFP_MaxList, A_MANFP_index_List[0:]))      
                        
except:
    pass

df_A_Pair_MANF_P = pd.DataFrame(A_Pair_MANFP_List)
df_A_Pair_MANF_P.columns = ['A_MANF_P', 'DATETIME']
df_A_Pair_MANF_P.set_index('DATETIME', inplace=True)

print('Done', str(datetime.datetime.now()))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Done 2020-07-13 09:55:00.309539


In [8]:
#Remove the 'No Good Data For Calculation' values using for loop
for x in range(0, len(df_comb['A_t_CWSH1'])):
    if df_comb['A_t_CWSH1'][x] =='[-11059] No Good Data For Calculation':
            df_comb['A_t_CWSH1'][x] = 0


#CWSH1
##wORKING as of 2020.01.15

keyslist = list(ft_cyc_lim.keys()) #creates a list of the ft_cyc_lim keys so that we can iterate later

A_t_CWSH1_Maxlist = list()
A_t_CWSH1_Maxlist[:] = []

A_CWSH1_index_List=list()      
A_CWSH1_index_List[:] = []

A_Pair_CWSH1_List =list() 
A_Pair_CWSH1_List[:] = []
try:         
    for i in range(0, len(keyslist)):
        A_t_CWSH1_val_List=list() 
        A_t_CWSH1_val_List[:] = []
        for j in range(keyslist[i], ft_cyc_lim[keyslist[i]]):               
            A_t_CWSH1 = df_comb['A_t_CWSH1'][j]         
            A_t_CWSH1_val_List.append(A_t_CWSH1)
        A_t_CWSH1_Maxlist.append(sorted(A_t_CWSH1_val_List)[-1])
        A_CWSH1_index_List.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))
    A_Pair_CWSH1_List = list(zip(A_t_CWSH1_Maxlist, A_CWSH1_index_List[0:]))      
                        
except:
    pass
df_A_Pair_CWSH1 = pd.DataFrame(A_Pair_CWSH1_List)
df_A_Pair_CWSH1.columns = ['A_CWSH1', 'DATETIME']
df_A_Pair_CWSH1.set_index('DATETIME', inplace=True)

print('Done', str(datetime.datetime.now()))

Done 2020-07-13 09:55:01.265669


In [9]:
#CWSH2
#Remove the 'No Good Data For Calculation' values using for loop
for x in range(0, len(df_comb['A_t_CWSH2'])):
    if df_comb['A_t_CWSH2'][x] =='[-11059] No Good Data For Calculation':
            df_comb['A_t_CWSH2'][x] = 0

##wORKING as of 2020.01.15
keyslist = list(ft_cyc_lim.keys()) #creates a list of the ft_cyc_lim keys so that we can iterate later

A_t_CWSH2_Maxlist = list()
A_t_CWSH2_Maxlist[:] = []

A_CWSH2_index_List=list()      
A_CWSH2_index_List[:] = []

A_Pair_CWSH2_List =list() 
A_Pair_CWSH2_List[:] = []
try:         
    for i in range(0, len(keyslist)):
        A_t_CWSH2_val_List=list() 
        A_t_CWSH2_val_List[:] = []
        for j in range(keyslist[i], ft_cyc_lim[keyslist[i]]):               
            A_t_CWSH2 = df_comb['A_t_CWSH2'][j]         
            A_t_CWSH2_val_List.append(A_t_CWSH2)
        A_t_CWSH2_Maxlist.append(sorted(A_t_CWSH2_val_List)[-1])
        A_CWSH2_index_List.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))
    A_Pair_CWSH2_List = list(zip(A_t_CWSH2_Maxlist, A_CWSH2_index_List[0:]))      
                        
except:
    pass
  
df_A_Pair_CWSH2 = pd.DataFrame(A_Pair_CWSH2_List)
df_A_Pair_CWSH2.columns = ['A_CWSH2', 'DATETIME']
df_A_Pair_CWSH2.set_index('DATETIME', inplace=True)
print('Done', str(datetime.datetime.now()))

Done 2020-07-13 09:55:02.212872


In [10]:
#Pressing_1

#Remove the 'No Good Data For Calculation' values using for loop
for x in range(0, len(df_comb['A_t_PRESS1'])):
    if df_comb['A_t_PRESS1'][x] =='[-11059] No Good Data For Calculation':
            df_comb['A_t_PRESS1'][x] = 0

##wORKING as of 2020.01.15
keyslist = list(ft_cyc_lim.keys()) #creates a list of the ft_cyc_lim keys so that we can iterate later

A_t_Press1_Maxlist = list()
A_t_Press1_Maxlist[:] = []

A_t_Press1_index_List=list()      
A_t_Press1_index_List[:] = []

A_Pair_A_t_Press1_List =list() 
A_Pair_A_t_Press1_List[:] = []
try:         
    for i in range(0, len(keyslist)):
        A_t_Press1_val_List=list() 
        A_t_Press1_val_List[:] = []
        for j in range(keyslist[i], ft_cyc_lim[keyslist[i]]):               
            A_t_Press1 = df_comb['A_t_PRESS1'][j]         
            A_t_Press1_val_List.append(A_t_Press1)
        A_t_Press1_Maxlist.append(sorted(A_t_Press1_val_List)[-1])
        A_t_Press1_index_List.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))
    A_Pair_Press1_List = list(zip(A_t_Press1_Maxlist, A_t_Press1_index_List[0:]))      
                        
except:
    pass
df_A_Pair_PRESS1_t = pd.DataFrame(A_Pair_Press1_List)
df_A_Pair_PRESS1_t.columns = ['A_PRESS1_t', 'DATETIME']
df_A_Pair_PRESS1_t.set_index('DATETIME', inplace=True)
print('Done', str(datetime.datetime.now()))

Done 2020-07-13 09:55:03.368276


In [11]:
#Pressing_2

#Remove the 'No Good Data For Calculation' values using for loop
for x in range(0, len(df_comb['A_t_PRESS2'])):
    if df_comb['A_t_PRESS2'][x] =='[-11059] No Good Data For Calculation':
            df_comb['A_t_PRESS2'][x] = 0

##wORKING as of 2020.01.15
keyslist = list(ft_cyc_lim.keys()) #creates a list of the ft_cyc_lim keys so that we can iterate later

A_t_Press2_Maxlist = list()
A_t_Press2_Maxlist[:] = []

A_t_Press2_index_List=list()      
A_t_Press2_index_List[:] = []

A_Pair_Press2_List =list() 
A_Pair_Press2_List[:] = []
try:         
    for i in range(0, len(keyslist)):
        A_t_Press2_val_List=list() 
        A_t_Press2_val_List[:] = []
        for j in range(keyslist[i], ft_cyc_lim[keyslist[i]]):               
            A_t_Press2 = df_comb['A_t_PRESS2'][j]         
            A_t_Press2_val_List.append(A_t_Press2)
        A_t_Press2_Maxlist.append(sorted(A_t_Press2_val_List)[-1])
        A_t_Press2_index_List.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))
    A_Pair_Press2_List = list(zip(A_t_Press2_Maxlist, A_t_Press2_index_List[0:]))      
                        
except:
    pass
  
df_A_Pair_PRESS2_t = pd.DataFrame(A_Pair_Press2_List)
df_A_Pair_PRESS2_t.columns = ['A_PRESS2_t', 'DATETIME']
df_A_Pair_PRESS2_t.set_index('DATETIME', inplace=True)
print('Done', str(datetime.datetime.now()))

Done 2020-07-13 09:55:05.028828


In [12]:
#A_PU26A_OP

#Remove the 'No Good Data For Calculation' values using for loop
for x in range(0, len(df_comb['PU26A_OP'])):
    if df_comb['PU26A_OP'][x] =='[-11059] No Good Data For Calculation':
            df_comb['PU26A_OP'][x] = 0

##Working as of 2020.01.15
A_PU26A_OP_Maxlist = list()
A_PU26A_OP_Maxlist[:] = []

A_PU26A_OP_index_List=list()      
A_PU26A_OP_index_List[:] = []

A_Pair_PU26A_OP_List =list() 
A_Pair_PU26A_OP_List[:] = []
try:         
    for i in range(0, len(keyslist)):
        A_PU26A_OP_val_List=list() 
        A_PU26A_OP_val_List[:] = []
        for j in range(keyslist[i], ft_cyc_lim[keyslist[i]]):               
            A_PU26A_OP = df_comb['PU26A_OP'][j]      
            A_PU26A_OP_val_List.append(A_PU26A_OP)
        A_PU26A_OP_Maxlist.append(sorted(A_PU26A_OP_val_List)[-1])
        A_PU26A_OP_index_List.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))
    A_Pair_PU26A_OP_List = list(zip(A_PU26A_OP_Maxlist, A_PU26A_OP_index_List[0:]))          
except:
    pass
df_A_Pair_PU26A_OP = pd.DataFrame(A_Pair_PU26A_OP_List)
df_A_Pair_PU26A_OP.columns = ['A_PU26A_OP', 'DATETIME']
df_A_Pair_PU26A_OP.set_index('DATETIME', inplace=True)
print('Done', str(datetime.datetime.now()))

Done 2020-07-13 09:55:06.536837


In [13]:
#A_PU27A_OP
##Working as of 2020.01.15
keyslist = list(ft_cyc_lim.keys()) #creates a list of the ft_cyc_lim keys so that we can iterate later

A_PU27A_OP_Maxlist = list()
A_PU27A_OP_Maxlist[:] = []

A_PU27A_OP_index_List=list()      
A_PU27A_OP_index_List[:] = []

A_Pair_PU27A_OP_List =list() 
A_Pair_PU27A_OP_List[:] = []
try:         
    for i in range(0, len(keyslist)):
        A_PU27A_OP_val_List=list() 
        A_PU27A_OP_val_List[:] = []
        for j in range(keyslist[i], ft_cyc_lim[keyslist[i]]):               
            A_PU27A_OP = df_comb['PU27A_OP'][j]      
            A_PU27A_OP_val_List.append(A_PU27A_OP)
        A_PU27A_OP_Maxlist.append(sorted(A_PU27A_OP_val_List)[-1])
        A_PU27A_OP_index_List.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))
    A_Pair_PU27A_OP_List = list(zip(A_PU27A_OP_Maxlist, A_PU27A_OP_index_List[0:]))          
except:
    pass
  
df_A_Pair_PU27A_OP = pd.DataFrame(A_Pair_PU27A_OP_List)
df_A_Pair_PU27A_OP.columns = ['A_PU27A_OP', 'DATETIME']
df_A_Pair_PU27A_OP.set_index('DATETIME', inplace=True)
print('Done', str(datetime.datetime.now()))

Done 2020-07-13 09:55:07.518718


In [14]:
#this code replaces the 'No Good Data' from 106PU26A_CUR column
for x in range(0, len(df_comb['106PU26A_CUR'])):
    if df_comb['106PU26A_CUR'][x] =='[-11059] No Good Data For Calculation':
            df_comb['106PU26A_CUR'][x] = 0
#A_PU26A_CUR
##Working as of 2020.01.15
keyslist = list(ft_cyc_lim.keys()) #creates a list of the ft_cyc_lim keys so that we can iterate later

A_PU26A_CUR_Maxlist = list()
A_PU26A_CUR_Maxlist[:] = []

A_PU26A_CUR_index_List=list()      
A_PU26A_CUR_index_List[:] = []

A_Pair_PU26A_CUR_List =list() 
A_Pair_PU26A_CUR_List[:] = []
try:         
    for i in range(0, len(keyslist)):
        A_PU26A_CUR_val=list() 
        A_PU26A_CUR_val[:] = []
        for j in range(keyslist[i], ft_cyc_lim[keyslist[i]]):               
            A_PU26A_CUR = df_comb['106PU26A_CUR'][j]      
            A_PU26A_CUR_val.append(A_PU26A_CUR)
        A_PU26A_CUR_Maxlist.append(sorted(A_PU26A_CUR_val)[-1])
        A_PU26A_CUR_index_List.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))
    A_Pair_PU26A_CUR_List = list(zip(A_PU26A_CUR_Maxlist, A_PU26A_CUR_index_List[0:]))          
except:
    pass

df_A_Pair_PU26A_CUR = pd.DataFrame(A_Pair_PU26A_CUR_List)
df_A_Pair_PU26A_CUR.columns = ['A_PU26A_CUR', 'DATETIME']
df_A_Pair_PU26A_CUR.set_index('DATETIME', inplace=True)
print('Done', str(datetime.datetime.now()))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Done 2020-07-13 09:55:11.735809


In [15]:
#this code replaces the 'No Good Data' from 106PU26A_CUR column
for x in range(0, len(df_comb['106PU27A_CUR'])):
    if df_comb['106PU27A_CUR'][x] =='[-11059] No Good Data For Calculation':
            df_comb['106PU27A_CUR'][x] = 0
#A_PU27A_CUR
##Working as of 2020.01.15 after a million trials!!!!!!!!!! mabuak na akong utok!!!
keyslist = list(ft_cyc_lim.keys()) #creates a list of the ft_cyc_lim keys so that we can iterate later

A_PU27A_CUR_Maxlist = list()
A_PU27A_CUR_Maxlist[:] = []

A_PU27A_CUR_index_List=list()      
A_PU27A_CUR_index_List[:] = []

A_Pair_PU27A_CUR_List =list() 
A_Pair_PU27A_CUR_List[:] = []
try:         
    for i in range(0, len(keyslist)):
        A_PU27A_CUR_val=list() 
        A_PU27A_CUR_val[:] = []
        for j in range(keyslist[i], ft_cyc_lim[keyslist[i]]):               
            A_PU27A_CUR = df_comb['106PU27A_CUR'][j]      
            A_PU27A_CUR_val.append(A_PU27A_CUR)
        A_PU27A_CUR_Maxlist.append(sorted(A_PU27A_CUR_val)[-1])
        A_PU27A_CUR_index_List.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))
    A_Pair_PU27A_CUR_List = list(zip(A_PU27A_CUR_Maxlist, A_PU27A_CUR_index_List[0:]))          
except:
    pass
  
df_A_Pair_PU27A_CUR = pd.DataFrame(A_Pair_PU27A_CUR_List)
df_A_Pair_PU27A_CUR.columns = ['A_PU27A_CUR', 'DATETIME']
df_A_Pair_PU27A_CUR.set_index('DATETIME', inplace=True)
print('Done', str(datetime.datetime.now()))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Done 2020-07-13 09:55:13.698206


In [16]:
#A_N2 Dry
##Working as of 2020-01-15 17:26:09.736327
A_Pair_N2_Dry_List = list()
A_Pair_N2_Dry_List[:] = []
A_t_N2_Dry_indexList = list()
A_t_N2_Dry_indexList[:] = []
A_t_N2_Dry_val = list()
A_t_N2_Dry_val[:] = []
try:         
    for i in range(0, len(keyslist)):   
        A_t_N2_Dry = df_comb['A_t_DRY'][ft_cyc_lim[keyslist[i]]]
        A_t_N2_Dry_val.append(A_t_N2_Dry)
        A_t_N2_Dry_indexList.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))
    
    A_Pair_N2_Dry_List = list(zip(A_t_N2_Dry_val, A_t_N2_Dry_indexList[0:]))          
except:
    pass

df_A_Pair_N2DRY = pd.DataFrame(A_Pair_N2_Dry_List)
df_A_Pair_N2DRY.columns = ['A_N2DRY', 'DATETIME']
df_A_Pair_N2DRY.set_index('DATETIME', inplace=True)
print('Done', str(datetime.datetime.now()))

Done 2020-07-13 09:55:13.736104


In [17]:
dfn = df_A_Pair_CWP_1.join(df_A_Pair_CWP_2, how='outer')
dfn1 = df_A_Pair_Filt_t.join(df_A_Pair_MANF_P, how='outer')
dfn2 = df_A_Pair_CWSH1.join(df_A_Pair_CWSH2, how='outer')
dfn3 = df_A_Pair_PRESS1_t.join(df_A_Pair_PRESS2_t, how='outer')
dfn4 = df_A_Pair_PU26A_OP.join(df_A_Pair_PU27A_OP, how='outer')
dfn5 = df_A_Pair_PU26A_CUR.join(df_A_Pair_PU27A_CUR, how='outer')
dfn6 = df_A_Pair_N2DRY.join(dfn, how='outer')
dfn7 = dfn1.join(dfn2, how='outer')
dfn8 = dfn3.join(dfn4, how='outer')
dfn9 = dfn5.join(dfn6, how='outer')
dfn10 = dfn7.join(dfn8, how='outer')
dfn11 = dfn9.join(dfn10, how='outer')
df_PressureFilter_Param = dfn11
print('Done', str(datetime.datetime.now()))

Done 2020-07-13 09:55:19.867304


In [18]:
df_PressureFilter_Param.head()

,A_PU26A_CUR,A_PU27A_CUR,A_N2DRY,A_CWP_1,A_CWP_2,A_FILT_T,A_MANF_P,A_CWSH1,A_CWSH2,A_PRESS1_t,A_PRESS2_t,A_PU26A_OP,A_PU27A_OP
DATETIME,,,,,,,,,,,,,
2020-01-01 00:21:00 to 2020-01-01 00:52:00,48.755524,49.685760,235.0,0.83,0.79,568.0,0.19,155,185,185,170,60,60
2020-01-01 00:54:00 to 2020-01-01 01:27:00,49.186447,50.321880,235.0,0.83,0.83,683.0,0.22,155,185,185,170,60,60
2020-01-01 01:30:00 to 2020-01-01 02:03:00,47.866322,49.295879,215.0,0.87,0.83,777.0,0.19,165,195,185,170,60,60
2020-01-01 02:23:00 to 2020-01-01 02:54:00,48.256203,49.617363,215.0,0.83,0.82,686.0,0.16,165,195,175,170,60,60
2020-01-01 02:56:00 to 2020-01-01 03:27:00,48.741840,49.836239,215.0,0.83,0.83,681.0,0.15,165,195,175,170,60,60


In [ ]:
dfn11.to_excel(r'C:\Users\v.t.flores\Documents\106FT02A_data_JAN2020.xlsx')

# SELECT THE BRAVO PARAMETERS

In [18]:
#exc7
#finding peaks FT for B_filtration time

df_comb['B_t_FEED'] = pd.to_numeric(df_comb['B_t_FEED'], errors='coerce')
a = np.diff(np.sign(np.diff(df_comb['B_t_FEED']))).nonzero()[0] + 1               # local min & max
b = (np.diff(np.sign(np.diff(df_comb['B_t_FEED']))) > 0).nonzero()[0] + 1         # local min
c = (np.diff(np.sign(np.diff(df_comb['B_t_FEED']))) < 0).nonzero()[0] + 1         # local max
# +1 due to the fact that diff reduces the original index number


#finding peaks FT for A_N2 Dry time

df_comb['B_t_DRY'] = pd.to_numeric(df_comb['B_t_DRY'], errors='coerce')
d = np.diff(np.sign(np.diff(df_comb['B_t_DRY']))).nonzero()[0] + 1               # local min & max
e = (np.diff(np.sign(np.diff(df_comb['B_t_DRY']))) > 0).nonzero()[0] + 1         # local min
f = (np.diff(np.sign(np.diff(df_comb['B_t_DRY']))) < 0).nonzero()[0] + 1         # local max
# +1 due to the fact that diff reduces the original index number

#Use b as the starting index and f as the ending index for one full cylcle of Filtration to N2 Drying

#-----------------------------------------------------------------------------------------------------------------------

#From the peaks computed in previous code, the index range for each cycle is stored in a dictionary called ft_cyc_lim
try:
    ft_cyc_lim = {}

    for i in range(0, len(b)):
        for j in range(0, len(f)):
            if f[j] > b[i] and f[j] < b[i+1]:
                if f[j+1]<b[i+1]:
                    cyc_start = b[i]
                    cyc_end = f[j+1]
                    ft_cyc_lim.setdefault(cyc_start,cyc_end)

except IndexError:
    print('\nDone')
#------------------------------------------------------------     

print('Done', str(datetime.datetime.now()))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in sign
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in sign
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in greater
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in sign
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in less
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in sign
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in sign
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: in


Done
Done 2020-01-27 14:18:12.279910


In [20]:
keyslist = list(ft_cyc_lim.keys()) #creates a list of the ft_cyc_lim keys so that we can iterate later
#Remove the 'No Good Data For Calculation' values using for loop
for x in range(0, len(df_comb['FT02B_CWP'])):
    if df_comb['FT02B_CWP'][x] =='[-11059] No Good Data For Calculation':
            df_comb['FT02B_CWP'][x] = 0

#code for getting the CWP for CakeWash1 and CakeWash2
##Working as of 1/15/2020
B_CWP_1new = list()
B_CWP_1new[:] = []

B_CWP_1_indexList = list()
B_CWP_1_indexList[:] = []

B_CWP_2new = list()
B_CWP_2new[:] = []

B_CWP_2_indexList = list()
B_CWP_2_indexList[:] = []

B_CWP_1_inner = list()
B_CWP_1_inner[:] = []

B_CWP_2_inner = list()
B_CWP_2_inner[:] = []

B_Pair_CWP_1_List = list()
B_Pair_CWP_2_List = list()

try:
    for i in range(0, len(keyslist)):
        B_CWPList_i = list()
        B_CWPList_i[:] = []
        d = list()
        d[:] = []
        
        B_CWPList_i = list(np.arange(keyslist[i],ft_cyc_lim[keyslist[i]]))
        d = (np.diff(np.sign(np.diff(df_comb['FT02B_CWP'].iloc[keyslist[i]:ft_cyc_lim[keyslist[i]]])))< 0).nonzero()[0] + 1
        B_CWP_for_sort = list()
        B_CWP_for_sort[:] = [] 
        
        B_CWP_1_indexList.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))
        B_CWP_2_indexList.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))
        
        if d.size != 0:
            for i in range(0, len(d)):
                CWP_val_shortlist = df_comb['FT02B_CWP'].iloc[B_CWPList_i[d[i]]]
                B_CWP_for_sort.append(CWP_val_shortlist)

            B_CWP_1 = sorted(B_CWP_for_sort)[len(B_CWP_for_sort)-1]
            B_CWP_1_inner.append(B_CWP_1)

            B_CWP_2 = sorted(B_CWP_for_sort)[len(B_CWP_for_sort)-2]
            B_CWP_2_inner.append(B_CWP_2)
        if d.size == 0:
            B_CWP_1_inner.append(0)
            B_CWP_2_inner.append(0)

    B_Pair_CWP_1_List = list(zip(B_CWP_1_inner, B_CWP_1_indexList[0:]))
    B_Pair_CWP_2_List = list(zip(B_CWP_2_inner, B_CWP_2_indexList[0:]))
            
    
except IndexError:
    print('IndexError')

df_B_Pair_CWP_1 = pd.DataFrame(B_Pair_CWP_1_List)
df_B_Pair_CWP_1.columns = ['B_CWP_1', 'DATETIME']
df_B_Pair_CWP_1.set_index('DATETIME', inplace=True)
    
df_B_Pair_CWP_2 = pd.DataFrame(B_Pair_CWP_2_List)
df_B_Pair_CWP_2.columns = ['B_CWP_2', 'DATETIME']
df_B_Pair_CWP_2.set_index('DATETIME', inplace=True)
   
    
print('Done', str(datetime.datetime.now()))

Done 2020-01-27 14:20:15.803884


In [21]:
# This code will use the index range (or limits) to create a list of values: feedtime range, cake wash time range,
# then selects the max value as the parameter value for that range

#FILTRATION TIME
keyslist = list(ft_cyc_lim.keys()) #creates a list of the ft_cyc_lim keys so that we can iterate later
#For loop Dictionary names 
B_Filt_MaxList= list()
B_Filt_MaxList[:] = []
B_feedt_Val_List = list()
B_feedt_Val_List[:] = []
B_feedt_index_List = list()
B_feedt_index_List[:] = []
B_Pair_Filt_List = list()
B_Pair_Filt_List[:] = []
try:         
    for i in range(0, len(keyslist)):
        B_feedt_Val_List = list()
        B_feedt_Val_List[:] = []
        for j in range(keyslist[i], ft_cyc_lim[keyslist[i]]):
            B_feedt = df_comb['B_t_FEED'][j]
            B_feedt_Val_List.append(B_feedt)
        B_Filt_MaxList.append(sorted(B_feedt_Val_List)[-1])
        B_feedt_index_List.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))  
    B_Pair_Filt_List = list(zip(B_Filt_MaxList, B_feedt_index_List[0:]))      
except:
    pass

df_B_Pair_Filt_t = pd.DataFrame(B_Pair_Filt_List)
df_B_Pair_Filt_t.columns = ['B_FILT_T', 'DATETIME']
df_B_Pair_Filt_t.set_index('DATETIME', inplace=True)

print('Done', str(datetime.datetime.now()))


Done 2020-01-27 14:20:19.067941


In [22]:
#MANIFOLD PRESSURE_B
keyslist = list(ft_cyc_lim.keys()) #creates a list of the ft_cyc_lim keys so that we can iterate later
#For loop Dictionary names 
B_MANFP_MaxList = list()
B_MANFP_MaxList[:] = []
B_Pair_MANFP_List =list() 
B_Pair_MANFP_List[:] = []
B_MANFP_index_List=list()      
B_MANFP_index_List[:] = []
try:         
    for i in range(0, len(keyslist)):
        B_MANFP_val_List=list() 
        B_MANFP_val_List[:] = []
        for j in range(keyslist[i], ft_cyc_lim[keyslist[i]]):               
            #Gets A manifold pressure of the cycle
            B_MANFP = df_comb['FT02B_MANFP'][j]         
            B_MANFP_val_List.append(B_MANFP)
        B_MANFP_MaxList.append(sorted(B_MANFP_val_List)[-1])
        B_MANFP_index_List.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))
    B_Pair_MANFP_List = list(zip(B_MANFP_MaxList, B_MANFP_index_List[0:]))      
                        
except:
    pass

df_B_Pair_MANF_P = pd.DataFrame(B_Pair_MANFP_List)
df_B_Pair_MANF_P.columns = ['B_MANF_P', 'DATETIME']
df_B_Pair_MANF_P.set_index('DATETIME', inplace=True)

print('Done', str(datetime.datetime.now()))

Done 2020-01-27 14:20:20.622203


In [23]:
#CWSH1
##wORKING as of 2020.01.15

keyslist = list(ft_cyc_lim.keys()) #creates a list of the ft_cyc_lim keys so that we can iterate later

B_t_CWSH1_Maxlist = list()
B_t_CWSH1_Maxlist[:] = []

B_CWSH1_index_List=list()      
B_CWSH1_index_List[:] = []

B_Pair_CWSH1_List =list() 
B_Pair_CWSH1_List[:] = []
try:         
    for i in range(0, len(keyslist)):
        B_t_CWSH1_val_List=list() 
        B_t_CWSH1_val_List[:] = []
        for j in range(keyslist[i], ft_cyc_lim[keyslist[i]]):               
            B_t_CWSH1 = df_comb['B_t_CWSH1'][j]         
            B_t_CWSH1_val_List.append(B_t_CWSH1)
        B_t_CWSH1_Maxlist.append(sorted(B_t_CWSH1_val_List)[-1])
        B_CWSH1_index_List.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))
    B_Pair_CWSH1_List = list(zip(B_t_CWSH1_Maxlist, B_CWSH1_index_List[0:]))      
                        
except:
    pass
df_B_Pair_CWSH1 = pd.DataFrame(B_Pair_CWSH1_List)
df_B_Pair_CWSH1.columns = ['B_CWSH1', 'DATETIME']
df_B_Pair_CWSH1.set_index('DATETIME', inplace=True)

print('Done', str(datetime.datetime.now()))
#CWSH2
##wORKING as of 2020.01.15
keyslist = list(ft_cyc_lim.keys()) #creates a list of the ft_cyc_lim keys so that we can iterate later

B_t_CWSH2_Maxlist = list()
B_t_CWSH2_Maxlist[:] = []

B_CWSH2_index_List=list()      
B_CWSH2_index_List[:] = []

B_Pair_CWSH2_List =list() 
B_Pair_CWSH2_List[:] = []
try:         
    for i in range(0, len(keyslist)):
        B_t_CWSH2_val_List=list() 
        B_t_CWSH2_val_List[:] = []
        for j in range(keyslist[i], ft_cyc_lim[keyslist[i]]):               
            B_t_CWSH2 = df_comb['B_t_CWSH2'][j]         
            B_t_CWSH2_val_List.append(B_t_CWSH2)
        B_t_CWSH2_Maxlist.append(sorted(B_t_CWSH2_val_List)[-1])
        B_CWSH2_index_List.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))
    B_Pair_CWSH2_List = list(zip(B_t_CWSH2_Maxlist, B_CWSH2_index_List[0:]))      
                        
except:
    pass
  
df_B_Pair_CWSH2 = pd.DataFrame(B_Pair_CWSH2_List)
df_B_Pair_CWSH2.columns = ['B_CWSH2', 'DATETIME']
df_B_Pair_CWSH2.set_index('DATETIME', inplace=True)
print('Done', str(datetime.datetime.now()))
#Pressing_1
##wORKING as of 2020.01.15
keyslist = list(ft_cyc_lim.keys()) #creates a list of the ft_cyc_lim keys so that we can iterate later

B_t_Press1_Maxlist = list()
B_t_Press1_Maxlist[:] = []

B_t_Press1_index_List=list()      
B_t_Press1_index_List[:] = []

B_Pair_B_t_Press1_List =list() 
B_Pair_B_t_Press1_List[:] = []
try:         
    for i in range(0, len(keyslist)):
        B_t_Press1_val_List=list() 
        B_t_Press1_val_List[:] = []
        for j in range(keyslist[i], ft_cyc_lim[keyslist[i]]):               
            B_t_Press1 = df_comb['B_t_PRESS1'][j]         
            B_t_Press1_val_List.append(B_t_Press1)
        B_t_Press1_Maxlist.append(sorted(B_t_Press1_val_List)[-1])
        B_t_Press1_index_List.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))
    B_Pair_Press1_List = list(zip(B_t_Press1_Maxlist, B_t_Press1_index_List[0:]))      
                        
except:
    pass
df_B_Pair_PRESS1_t = pd.DataFrame(B_Pair_Press1_List)
df_B_Pair_PRESS1_t.columns = ['B_PRESS1_t', 'DATETIME']
df_B_Pair_PRESS1_t.set_index('DATETIME', inplace=True)
print('Done', str(datetime.datetime.now()))
#Pressing_2
##wORKING as of 2020.01.15
keyslist = list(ft_cyc_lim.keys()) #creates a list of the ft_cyc_lim keys so that we can iterate later

B_t_Press2_Maxlist = list()
B_t_Press2_Maxlist[:] = []

B_t_Press2_index_List=list()      
B_t_Press2_index_List[:] = []

B_Pair_Press2_List =list() 
B_Pair_Press2_List[:] = []
try:         
    for i in range(0, len(keyslist)):
        B_t_Press2_val_List=list() 
        B_t_Press2_val_List[:] = []
        for j in range(keyslist[i], ft_cyc_lim[keyslist[i]]):               
            B_t_Press2 = df_comb['B_t_PRESS2'][j]         
            B_t_Press2_val_List.append(B_t_Press2)
        B_t_Press2_Maxlist.append(sorted(B_t_Press2_val_List)[-1])
        B_t_Press2_index_List.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))
    B_Pair_Press2_List = list(zip(B_t_Press2_Maxlist, B_t_Press2_index_List[0:]))      
                        
except:
    pass
  
df_B_Pair_PRESS2_t = pd.DataFrame(B_Pair_Press2_List)
df_B_Pair_PRESS2_t.columns = ['B_PRESS2_t', 'DATETIME']
df_B_Pair_PRESS2_t.set_index('DATETIME', inplace=True)
print('Done', str(datetime.datetime.now()))

#B_PU26B_OP
##Working as of 2020.01.15
B_PU26B_OP_Maxlist = list()
B_PU26B_OP_Maxlist[:] = []

B_PU26B_OP_index_List=list()      
B_PU26B_OP_index_List[:] = []

B_Pair_PU26B_OP_List =list() 
B_Pair_PU26B_OP_List[:] = []
try:         
    for i in range(0, len(keyslist)):
        B_PU26B_OP_val_List=list() 
        B_PU26B_OP_val_List[:] = []
        for j in range(keyslist[i], ft_cyc_lim[keyslist[i]]):               
            B_PU26B_OP = df_comb['PU26B_OP'][j]      
            B_PU26B_OP_val_List.append(B_PU26B_OP)
        B_PU26B_OP_Maxlist.append(sorted(B_PU26B_OP_val_List)[-1])
        B_PU26B_OP_index_List.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))
    B_Pair_PU26B_OP_List = list(zip(B_PU26B_OP_Maxlist, B_PU26B_OP_index_List[0:]))          
except:
    pass
df_B_Pair_PU26B_OP = pd.DataFrame(B_Pair_PU26B_OP_List)
df_B_Pair_PU26B_OP.columns = ['B_PU26B_OP', 'DATETIME']
df_B_Pair_PU26B_OP.set_index('DATETIME', inplace=True)
print('Done', str(datetime.datetime.now()))

#B_PU27B_OP
##Working as of 2020.01.15
keyslist = list(ft_cyc_lim.keys()) #creates a list of the ft_cyc_lim keys so that we can iterate later

B_PU27B_OP_Maxlist = list()
B_PU27B_OP_Maxlist[:] = []

B_PU27B_OP_index_List=list()      
B_PU27B_OP_index_List[:] = []

B_Pair_PU27B_OP_List =list() 
B_Pair_PU27B_OP_List[:] = []
try:         
    for i in range(0, len(keyslist)):
        B_PU27B_OP_val_List=list() 
        B_PU27B_OP_val_List[:] = []
        for j in range(keyslist[i], ft_cyc_lim[keyslist[i]]):               
            B_PU27B_OP = df_comb['PU27B_OP'][j]      
            B_PU27B_OP_val_List.append(B_PU27B_OP)
        B_PU27B_OP_Maxlist.append(sorted(B_PU27B_OP_val_List)[-1])
        B_PU27B_OP_index_List.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))
    B_Pair_PU27B_OP_List = list(zip(B_PU27B_OP_Maxlist, B_PU27B_OP_index_List[0:]))          
except:
    pass
  
df_B_Pair_PU27B_OP = pd.DataFrame(B_Pair_PU27B_OP_List)
df_B_Pair_PU27B_OP.columns = ['B_PU27B_OP', 'DATETIME']
df_B_Pair_PU27B_OP.set_index('DATETIME', inplace=True)
print('Done', str(datetime.datetime.now()))

#this code replaces the 'No Good Data' from 106PU26A_CUR column
for x in range(0, len(df_comb['106PU26B_CUR'])):
    if df_comb['106PU26B_CUR'][x] =='[-11059] No Good Data For Calculation':
            df_comb['106PU26B_CUR'][x] = 0
#B_PU26A_CUR
##Working as of 2020.01.15
keyslist = list(ft_cyc_lim.keys()) #creates a list of the ft_cyc_lim keys so that we can iterate later

B_PU26B_CUR_Maxlist = list()
B_PU26B_CUR_Maxlist[:] = []

B_PU26B_CUR_index_List=list()      
B_PU26B_CUR_index_List[:] = []

B_Pair_PU26B_CUR_List =list() 
B_Pair_PU26B_CUR_List[:] = []
try:         
    for i in range(0, len(keyslist)):
        B_PU26B_CUR_val=list() 
        B_PU26B_CUR_val[:] = []
        for j in range(keyslist[i], ft_cyc_lim[keyslist[i]]):               
            B_PU26B_CUR = df_comb['106PU26B_CUR'][j]      
            B_PU26B_CUR_val.append(B_PU26B_CUR)
        B_PU26B_CUR_Maxlist.append(sorted(B_PU26B_CUR_val)[-1])
        B_PU26B_CUR_index_List.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))
    B_Pair_PU26B_CUR_List = list(zip(B_PU26B_CUR_Maxlist, B_PU26B_CUR_index_List[0:]))          
except:
    pass

df_B_Pair_PU26B_CUR = pd.DataFrame(B_Pair_PU26B_CUR_List)
df_B_Pair_PU26B_CUR.columns = ['B_PU26B_CUR', 'DATETIME']
df_B_Pair_PU26B_CUR.set_index('DATETIME', inplace=True)
print('Done', str(datetime.datetime.now()))

#this code replaces the 'No Good Data' from 106PU26A_CUR column
for x in range(0, len(df_comb['106PU27B_CUR'])):
    if df_comb['106PU27B_CUR'][x] =='[-11059] No Good Data For Calculation':
            df_comb['106PU27B_CUR'][x] = 0
#B_PU27B_CUR
##Working as of 2020.01.15 after a million trials!!!!!!!!!! mabuak na akong utok!!!
keyslist = list(ft_cyc_lim.keys()) #creates a list of the ft_cyc_lim keys so that we can iterate later

B_PU27B_CUR_Maxlist = list()
B_PU27B_CUR_Maxlist[:] = []

B_PU27B_CUR_index_List=list()      
B_PU27B_CUR_index_List[:] = []

B_Pair_PU27B_CUR_List =list() 
B_Pair_PU27B_CUR_List[:] = []
try:         
    for i in range(0, len(keyslist)):
        B_PU27B_CUR_val=list() 
        B_PU27B_CUR_val[:] = []
        for j in range(keyslist[i], ft_cyc_lim[keyslist[i]]):               
            B_PU27B_CUR = df_comb['106PU27B_CUR'][j]      
            B_PU27B_CUR_val.append(B_PU27B_CUR)
        B_PU27B_CUR_Maxlist.append(sorted(B_PU27B_CUR_val)[-1])
        B_PU27B_CUR_index_List.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))
    B_Pair_PU27B_CUR_List = list(zip(B_PU27B_CUR_Maxlist, B_PU27B_CUR_index_List[0:]))          
except:
    pass
  
df_B_Pair_PU27B_CUR = pd.DataFrame(B_Pair_PU27B_CUR_List)
df_B_Pair_PU27B_CUR.columns = ['B_PU27B_CUR', 'DATETIME']
df_B_Pair_PU27B_CUR.set_index('DATETIME', inplace=True)
print('Done', str(datetime.datetime.now()))

#A_N2 Dry
##Working as of 2020-01-15 17:26:09.736327
B_Pair_N2_Dry_List = list()
B_Pair_N2_Dry_List[:] = []
B_t_N2_Dry_indexList = list()
B_t_N2_Dry_indexList[:] = []
B_t_N2_Dry_val = list()
B_t_N2_Dry_val[:] = []
try:         
    for i in range(0, len(keyslist)):   
        B_t_N2_Dry = df_comb['B_t_DRY'][ft_cyc_lim[keyslist[i]]]
        B_t_N2_Dry_val.append(B_t_N2_Dry)
        B_t_N2_Dry_indexList.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))
    
    B_Pair_N2_Dry_List = list(zip(B_t_N2_Dry_val, B_t_N2_Dry_indexList[0:]))          
except:
    pass

df_B_Pair_N2DRY = pd.DataFrame(B_Pair_N2_Dry_List)
df_B_Pair_N2DRY.columns = ['B_N2DRY', 'DATETIME']
df_B_Pair_N2DRY.set_index('DATETIME', inplace=True)
print('Done', str(datetime.datetime.now()))


Done 2020-01-27 14:20:22.074497
Done 2020-01-27 14:20:22.409600
Done 2020-01-27 14:20:22.744677
Done 2020-01-27 14:20:23.081804
Done 2020-01-27 14:20:23.416911
Done 2020-01-27 14:20:23.750020


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Done 2020-01-27 14:20:25.236047


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:216: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Done 2020-01-27 14:20:26.710102
Done 2020-01-27 14:20:26.735035


In [24]:
#Combining the dataframes for bravo parameters
dfnB = df_B_Pair_CWP_1.join(df_B_Pair_CWP_2, how='outer')
dfn1B = df_B_Pair_Filt_t.join(df_B_Pair_MANF_P, how='outer')
dfn2B = df_B_Pair_CWSH1.join(df_B_Pair_CWSH2, how='outer')
dfn3B = df_B_Pair_PRESS1_t.join(df_B_Pair_PRESS2_t, how='outer')
dfn4B = df_B_Pair_PU26B_OP.join(df_B_Pair_PU27B_OP, how='outer')
dfn5B = df_B_Pair_PU26B_CUR.join(df_B_Pair_PU27B_CUR, how='outer')
dfn6B = df_B_Pair_N2DRY.join(dfnB, how='outer')
dfn7B = dfn1B.join(dfn2B, how='outer')
dfn8B = dfn3B.join(dfn4B, how='outer')
dfn9B = dfn5B.join(dfn6B, how='outer')
dfn10B = dfn7B.join(dfn8B, how='outer')
dfn11B = dfn9B.join(dfn10B, how='outer')
df_PressureFilter_Param_B = dfn11B
print('Done', str(datetime.datetime.now()))

Done 2020-01-27 14:20:28.297482


In [26]:
dfn11B.to_excel(r'C:\Users\v.t.flores\Documents\106FT02B_data_JAN2020.xlsx')

# LABORATORY ANALYSIS RESULTS

In [29]:
import openpyxl
wb1 = openpyxl.load_workbook(r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\JANUARY 2020.xlsx')
#####This block of code appends the Analysis Results from December2019
FT02wb = wb1['106FT02AB']
#update for the targetmonth
currentmonthyr = datetime.date(2020,1,1)
print('Done')

Done


In [25]:
#-----------------------
#collectst the index for the date.day the sample was taken and puts it in the list Smpl_day_index
Smpl_day_index = list()
Smpl_day_index[:] = []
for i in range (1, FT02wb.max_row):
    if type(FT02wb['B'+ str(i)].value) == int:
        #print(str(i)+':'+str(TH01_UF_sheet['B'+ str(i)].value))
        Smpl_day_index.append(i)

#smpl_end_index is the last row index for the date.day the sample was taken
for i in range(1, FT02wb.max_row):
    if str(FT02wb['B'+str(i)].value).split(' ')[0] == 'Daily':
        smpl_end_index = i
        break

#fcn definition        
def filter_detrmin(cell_addrs):
    PF_regex2 = re.compile(r'[a-z0-8]', re.IGNORECASE| re.VERBOSE|re.DOTALL)
    filtername2 = PF_regex2.findall(str(cell_addrs))
    for i in range(0, len(filtername2)):
        if filtername2[i] != filtername2[-1]:
            if filtername2[i]+filtername2[i+1] == '2A':
                PFname = 'A'
                return PFname
            if filtername2[i]+filtername2[i+1] == '2B':
                PFname = 'B'
                return PFname      
#-----------------end of function defn-----------        

#fcn dfn
def time_cleaner(x):
    time_regex = re.compile(r'[0-8a-zA-Z:]', re.IGNORECASE| re.VERBOSE|re.DOTALL)
    timeact = time_regex.findall(x)
    newtime = ''.join(timeact)
    t = parse(x)
    new_str_time = str(t.hour)+':'+str(t.minute)+':'+str(t.second)
    return new_str_time
#------------------

#Pb
#Code for creating a dataframe for MS_A analysis result for Element Pb
A_Pb_datelist = list()
A_Pb_datelist[:] = []
B_Pb_datelist = list()
B_Pb_datelist[:] = []

A_Pb_list = list()
A_Pb_list[:] = []
B_Pb_list = list()
B_Pb_list[:] = []

A_Pb_list_pair = list()
A_Pb_list_pair[:] = []
B_Pb_list_pair = list()
B_Pb_list_pair[:] = []

for y in range(0, len(Smpl_day_index)):   
    
    if Smpl_day_index[y] != Smpl_day_index[-1]:      
        for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
            if FT02wb['D'+str(x)].value != None:    
                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'A':  
                    
                    A_date = str((currentmonthyr)-datetime.timedelta(1)+\
                                 +datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    
                    A_Pb_datelist.append(parse(A_date))  
                    A_Pb = FT02wb['T'+str(x)].value
                    A_Pb_list.append(A_Pb)
                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'B':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'B',(FT02wb['D'+str(x)].value))
                    B_date = str((currentmonthyr)-datetime.timedelta(1)+\
                                 datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(B_datelist)
                    B_Pb_datelist.append(parse(B_date)) 
                    B_Pb = FT02wb['T'+str(x)].value
                    B_Pb_list.append(B_Pb)
             
    if Smpl_day_index[y] == Smpl_day_index[-1]:
        for x in range(Smpl_day_index[y], smpl_end_index):
            if FT02wb['D'+str(x)].value != None:    
                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'A':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'A',(FT02wb['D'+str(x)].value))
                    A_date = str((currentmonthyr)-datetime.timedelta(1)+\
                                 datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(A_date) 
                    A_Pb_datelist.append(parse(A_date))
                    A_Pb = FT02wb['T'+str(x)].value
                    A_Pb_list.append(A_Pb)

                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'B':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'B',(FT02wb['D'+str(x)].value))
                    B_date = str((currentmonthyr)-datetime.timedelta(1)+\
                                 datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(B_date)
                    B_Pb_datelist.append(parse(B_date))
                    B_Pb = FT02wb['T'+str(x)].value
                    B_Pb_list.append(B_Pb)
#this part creates the DataFrame after creating a list pair

A_Pb_list_pair = list(zip(A_Pb_list, A_Pb_datelist[0:]))
A_Pb_df = pd.DataFrame(A_Pb_list_pair)
A_Pb_df.columns = ['MS_A_Pb', 'DATETIME'] ##check if list title is updated
A_Pb_df.set_index('DATETIME', inplace=True)
A_Pb_df = A_Pb_df.loc[~A_Pb_df.index.duplicated(keep = 'last')]

B_Pb_list_pair = list(zip(B_Pb_list, B_Pb_datelist[0:]))
B_Pb_df = pd.DataFrame(B_Pb_list_pair)
B_Pb_df.columns = ['MS_B_Pb', 'DATETIME'] ##check if list title is updated

B_Pb_df.set_index('DATETIME', inplace=True)
B_Pb_df = B_Pb_df.loc[~B_Pb_df.index.duplicated(keep = 'last')]
#-----End for Pb code


NameError: name 'FT02wb' is not defined

In [34]:
#Zn
#Code for creating a dataframe for MS_A analysis result for Element Zn
A_Zn_datelist = list()
A_Zn_datelist[:] = []
B_Zn_datelist = list()
B_Zn_datelist[:] = []

A_Zn_list = list()
A_Zn_list[:] = []
B_Zn_list = list()
B_Zn_list[:] = []

A_Zn_list_pair = list()
A_Zn_list_pair[:] = []
B_Zn_list_pair = list()
B_Zn_list_pair[:] = []

for y in range(0, len(Smpl_day_index)):   
    
    if Smpl_day_index[y] != Smpl_day_index[-1]:      
        for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
            if FT02wb['D'+str(x)].value != None:    
                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'A':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'A',(FT02wb['D'+str(x)].value))
                    A_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(parse(A_date))
                    A_Zn_datelist.append(parse(A_date))  
                    A_Zn = FT02wb['U'+str(x)].value
                    A_Zn_list.append(A_Zn)
                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'B':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'B',(FT02wb['D'+str(x)].value))
                    B_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(B_datelist)
                    B_Zn_datelist.append(parse(B_date)) 
                    B_Zn = FT02wb['U'+str(x)].value
                    B_Zn_list.append(B_Zn)

    if Smpl_day_index[y] == Smpl_day_index[-1]:
        for x in range(Smpl_day_index[y], smpl_end_index):
            if FT02wb['D'+str(x)].value != None:    
                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'A':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'A',(FT02wb['D'+str(x)].value))
                    A_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(A_date) 
                    A_Zn_datelist.append(parse(A_date))
                    A_Zn = FT02wb['U'+str(x)].value
                    A_Zn_list.append(A_Zn)

                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'B':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'B',(FT02wb['D'+str(x)].value))
                    B_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(B_date)
                    B_Zn_datelist.append(parse(B_date))
                    B_Zn = FT02wb['U'+str(x)].value
                    B_Zn_list.append(B_Zn)
#this part creates the DataFrame after creating a list pair

A_Zn_list_pair = list(zip(A_Zn_list, A_Zn_datelist[0:]))
A_Zn_df = pd.DataFrame(A_Zn_list_pair)
A_Zn_df.columns = ['MS_A_Zn', 'DATETIME'] ##check if list title is updated
A_Zn_df.set_index('DATETIME', inplace=True)
A_Zn_df = A_Zn_df.loc[~A_Zn_df.index.duplicated(keep = 'last')]


B_Zn_list_pair = list(zip(B_Zn_list, B_Zn_datelist[0:]))
B_Zn_df = pd.DataFrame(B_Zn_list_pair)
B_Zn_df.columns = ['MS_B_Zn', 'DATETIME'] ##check if list title is updated
B_Zn_df.set_index('DATETIME', inplace=True)
B_Zn_df = B_Zn_df.loc[~B_Zn_df.index.duplicated(keep = 'last')]
#-----End for Zn code

In [35]:


#Cu
#Code for creating a dataframe for MS_A analysis result for Element Cu
A_Cu_datelist = list()
A_Cu_datelist[:] = []
B_Cu_datelist = list()
B_Cu_datelist[:] = []

A_Cu_list = list()
A_Cu_list[:] = []
B_Cu_list = list()
B_Cu_list[:] = []

A_Cu_list_pair = list()
A_Cu_list_pair[:] = []
B_Cu_list_pair = list()
B_Cu_list_pair[:] = []

for y in range(0, len(Smpl_day_index)):   
    
    if Smpl_day_index[y] != Smpl_day_index[-1]:      
        for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
            if FT02wb['D'+str(x)].value != None:    
                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'A':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'A',(FT02wb['D'+str(x)].value))
                    A_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(parse(A_date))
                    A_Cu_datelist.append(parse(A_date))  
                    A_Cu = FT02wb['V'+str(x)].value
                    A_Cu_list.append(A_Cu)
                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'B':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'B',(FT02wb['D'+str(x)].value))
                    B_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(B_datelist)
                    B_Cu_datelist.append(parse(B_date)) 
                    B_Cu = FT02wb['V'+str(x)].value
                    B_Cu_list.append(B_Cu)

    if Smpl_day_index[y] == Smpl_day_index[-1]:
        for x in range(Smpl_day_index[y], smpl_end_index):
            if FT02wb['D'+str(x)].value != None:    
                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'A':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'A',(FT02wb['D'+str(x)].value))
                    A_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(A_date) 
                    A_Cu_datelist.append(parse(A_date))
                    A_Cu = FT02wb['V'+str(x)].value
                    A_Cu_list.append(A_Cu)

                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'B':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'B',(FT02wb['D'+str(x)].value))
                    B_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(B_date)
                    B_Cu_datelist.append(parse(B_date))
                    B_Cu = FT02wb['V'+str(x)].value
                    B_Cu_list.append(B_Cu)
#this part creates the DataFrame after creating a list pair

A_Cu_list_pair = list(zip(A_Cu_list, A_Cu_datelist[0:]))
A_Cu_df = pd.DataFrame(A_Cu_list_pair)
A_Cu_df.columns = ['MS_A_Cu', 'DATETIME'] ##check if list title is updated
A_Cu_df.set_index('DATETIME', inplace=True)
A_Cu_df = A_Cu_df.loc[~A_Cu_df.index.duplicated(keep = 'last')]

B_Cu_list_pair = list(zip(B_Cu_list, B_Cu_datelist[0:]))
B_Cu_df = pd.DataFrame(B_Cu_list_pair)
B_Cu_df.columns = ['MS_B_Cu', 'DATETIME'] ##check if list title is updated
B_Cu_df.set_index('DATETIME', inplace=True)
B_Cu_df = B_Cu_df.loc[~B_Cu_df.index.duplicated(keep = 'last')]

#-----End for Cu code

In [36]:
#Ni
#Code for creating a dataframe for MS_A analysis result for Element Ni
A_Ni_datelist = list()
A_Ni_datelist[:] = []
B_Ni_datelist = list()
B_Ni_datelist[:] = []

A_Ni_list = list()
A_Ni_list[:] = []
B_Ni_list = list()
B_Ni_list[:] = []

A_Ni_list_pair = list()
A_Ni_list_pair[:] = []
B_Ni_list_pair = list()
B_Ni_list_pair[:] = []

for y in range(0, len(Smpl_day_index)):   
    
    if Smpl_day_index[y] != Smpl_day_index[-1]:      
        for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
            if FT02wb['D'+str(x)].value != None:    
                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'A':
                    A_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    A_Ni_datelist.append(parse(A_date))  
                    A_Ni = FT02wb['W'+str(x)].value
                    A_Ni_list.append(A_Ni)
                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'B':
                    B_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                  
                    B_Ni_datelist.append(parse(B_date)) 
                    B_Ni = FT02wb['W'+str(x)].value
                    B_Ni_list.append(B_Ni)

    if Smpl_day_index[y] == Smpl_day_index[-1]:
        for x in range(Smpl_day_index[y], smpl_end_index):
            if FT02wb['D'+str(x)].value != None:    
                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'A':
                    A_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    A_Ni_datelist.append(parse(A_date))
                    A_Ni = FT02wb['W'+str(x)].value
                    A_Ni_list.append(A_Ni)

                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'B':
                    B_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    B_Ni_datelist.append(parse(B_date))
                    B_Ni = FT02wb['W'+str(x)].value
                    B_Ni_list.append(B_Ni)
#this part creates the DataFrame after creating a list pair

A_Ni_list_pair = list(zip(A_Ni_list, A_Ni_datelist[0:]))
A_Ni_df = pd.DataFrame(A_Ni_list_pair)
A_Ni_df.columns = ['MS_A_Ni', 'DATETIME'] ##check if list title is updated
A_Ni_df.set_index('DATETIME', inplace=True)
A_Ni_df = A_Ni_df.loc[~A_Ni_df.index.duplicated(keep = 'last')]

B_Ni_list_pair = list(zip(B_Ni_list, B_Ni_datelist[0:]))
B_Ni_df = pd.DataFrame(B_Ni_list_pair)
B_Ni_df.columns = ['MS_B_Ni', 'DATETIME'] ##check if list title is updated
B_Ni_df.set_index('DATETIME', inplace=True)
B_Ni_df = B_Ni_df.loc[~B_Ni_df.index.duplicated(keep = 'last')]

#-----End for Ni code

In [37]:
#Co
#Code for creating a dataframe for MS_A analysis result for Element Co
A_Co_datelist = list()
A_Co_datelist[:] = []
B_Co_datelist = list()
B_Co_datelist[:] = []

A_Co_list = list()
A_Co_list[:] = []
B_Co_list = list()
B_Co_list[:] = []

A_Co_list_pair = list()
A_Co_list_pair[:] = []
B_Co_list_pair = list()
B_Co_list_pair[:] = []

for y in range(0, len(Smpl_day_index)):   
    
    if Smpl_day_index[y] != Smpl_day_index[-1]:      
        for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
            if FT02wb['D'+str(x)].value != None:    
                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'A':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'A',(FT02wb['D'+str(x)].value))
                    A_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(parse(A_date))
                    A_Co_datelist.append(parse(A_date))  
                    A_Co = FT02wb['X'+str(x)].value



#Mg
#Mgde for creating a dataframe for MS_A analysis result for Element Mg
A_Mg_datelist = list()
A_Mg_datelist[:] = []
B_Mg_datelist = list()
B_Mg_datelist[:] = []

A_Mg_list = list()
A_Mg_list[:] = []
B_Mg_list = list()	
B_Mg_list[:] = []

A_Mg_list_pair = list()
A_Mg_list_pair[:] = []
B_Mg_list_pair = list()
B_Mg_list_pair[:] = []

for y in range(0, len(Smpl_day_index)):   
    
    if Smpl_day_index[y] != Smpl_day_index[-1]:      
        for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
            if FT02wb['D'+str(x)].value != None:    
                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'A':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'A',(FT02wb['D'+str(x)].value))
                    A_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(parse(A_date))
                    A_Mg_datelist.append(parse(A_date))  
                    A_Mg = FT02wb['AE'+str(x)].value
                    A_Mg_list.append(A_Mg)
                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'B':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'B',(FT02wb['D'+str(x)].value))
                    B_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(B_datelist)
                    B_Mg_datelist.append(parse(B_date)) 
                    B_Mg = FT02wb['AE'+str(x)].value
                    B_Mg_list.append(B_Mg)

    if Smpl_day_index[y] == Smpl_day_index[-1]:
        for x in range(Smpl_day_index[y], smpl_end_index):
            if FT02wb['D'+str(x)].value != None:    
                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'A':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'A',(FT02wb['D'+str(x)].value))
                    A_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(A_date) 
                    A_Mg_datelist.append(parse(A_date))
                    A_Mg = FT02wb['AE'+str(x)].value
                    A_Mg_list.append(A_Mg)

                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'B':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'B',(FT02wb['D'+str(x)].value))
                    B_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(B_date)
                    B_Mg_datelist.append(parse(B_date))
                    B_Mg = FT02wb['AE'+str(x)].value
                    B_Mg_list.append(B_Mg)
#this part creates the DataFrame after creating a list pair

A_Mg_list_pair = list(zip(A_Mg_list, A_Mg_datelist[0:]))
A_Mg_df = pd.DataFrame(A_Mg_list_pair)
A_Mg_df.columns = ['MS_A_Mg', 'DATETIME'] ##check if list title is updated
A_Mg_df.set_index('DATETIME', inplace=True)
A_Mg_df = A_Mg_df.loc[~A_Mg_df.index.duplicated(keep = 'last')]


B_Mg_list_pair = list(zip(B_Mg_list, B_Mg_datelist[0:]))
B_Mg_df = pd.DataFrame(B_Mg_list_pair)
B_Mg_df.columns = ['MS_B_Mg', 'DATETIME'] ##check if list title is updated
B_Mg_df.set_index('DATETIME', inplace=True)
B_Mg_df = B_Mg_df.loc[~B_Mg_df.index.duplicated(keep = 'last')]

#-----End for Mg code

#S
#Sde for creating a dataframe for MS_A analysis result for Element S
A_S_datelist = list()
A_S_datelist[:] = []
B_S_datelist = list()
B_S_datelist[:] = []

A_S_list = list()
A_S_list[:] = []	
B_S_list = list()	
B_S_list[:] = []

A_S_list_pair = list()
A_S_list_pair[:] = []
B_S_list_pair = list()
B_S_list_pair[:] = []

for y in range(0, len(Smpl_day_index)):   
    
    if Smpl_day_index[y] != Smpl_day_index[-1]:      
        for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
            if FT02wb['D'+str(x)].value != None:    
                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'A':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'A',(FT02wb['D'+str(x)].value))
                    A_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(parse(A_date))
                    A_S_datelist.append(parse(A_date))  
                    A_S = FT02wb['AF'+str(x)].value
                    A_S_list.append(A_S)
                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'B':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'B',(FT02wb['D'+str(x)].value))
                    B_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(B_datelist)
                    B_S_datelist.append(parse(B_date)) 
                    B_S = FT02wb['AF'+str(x)].value
                    B_S_list.append(B_S)

    if Smpl_day_index[y] == Smpl_day_index[-1]:
        for x in range(Smpl_day_index[y], smpl_end_index):
            if FT02wb['D'+str(x)].value != None:    
                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'A':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'A',(FT02wb['D'+str(x)].value))
                    A_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(A_date) 
                    A_S_datelist.append(parse(A_date))
                    A_S = FT02wb['AF'+str(x)].value
                    A_S_list.append(A_S)

                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'B':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'B',(FT02wb['D'+str(x)].value))
                    B_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(B_date)
                    B_S_datelist.append(parse(B_date))
                    B_S = FT02wb['AF'+str(x)].value
                    B_S_list.append(B_S)
#this part creates the DataFrame after creating a list pair

A_S_list_pair = list(zip(A_S_list, A_S_datelist[0:]))
A_S_df = pd.DataFrame(A_S_list_pair)
A_S_df.columns = ['MS_A_S', 'DATETIME'] ##check if list title is updated
A_S_df.set_index('DATETIME', inplace=True)
A_S_df = A_S_df.loc[~A_S_df.index.duplicated(keep = 'last')]


B_S_list_pair = list(zip(B_S_list, B_S_datelist[0:]))
B_S_df = pd.DataFrame(B_S_list_pair)
B_S_df.columns = ['MS_B_S', 'DATETIME'] ##check if list title is updated
B_S_df.set_index('DATETIME', inplace=True)
B_S_df = B_S_df.loc[~B_S_df.index.duplicated(keep = 'last')]

#-----End for S code

#S
#Sde for creating a dataframe for MS_A analysis result for Element S
A_h2o_datelist = list()
A_h2o_datelist[:] = []
B_h2o_datelist = list()
B_h2o_datelist[:] = []

A_h2o_list = list()
A_h2o_list[:] = []	
B_h2o_list = list()	
B_h2o_list[:] = []

A_h2o_list_pair = list()
A_h2o_list_pair[:] = []
B_h2o_list_pair = list()
B_h2o_list_pair[:] = []

for y in range(0, len(Smpl_day_index)):   
    
    if Smpl_day_index[y] != Smpl_day_index[-1]:      
        for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
            if FT02wb['D'+str(x)].value != None:    
                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'A':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'A',(FT02wb['D'+str(x)].value))
                    A_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(parse(A_date))
                    A_h2o_datelist.append(parse(A_date))  
                    A_h2o = FT02wb['AP'+str(x)].value
                    A_h2o_list.append(A_h2o)
                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'B':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'B',(FT02wb['D'+str(x)].value))
                    B_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(B_datelist)
                    B_h2o_datelist.append(parse(B_date)) 
                    B_h2o = FT02wb['AP'+str(x)].value
                    B_h2o_list.append(B_h2o)
             
    if Smpl_day_index[y] == Smpl_day_index[-1]:
        for x in range(Smpl_day_index[y], smpl_end_index):
            if FT02wb['D'+str(x)].value != None:    
                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'A':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'A',(FT02wb['D'+str(x)].value))
                    A_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(A_date) 
                    A_h2o_datelist.append(parse(A_date))
                    A_h2o = FT02wb['AP'+str(x)].value
                    B_h2o_list.append(A_h2o)

                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'B':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'B',(FT02wb['D'+str(x)].value))
                    B_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(B_date)
                    B_h2o_datelist.append(parse(B_date))
                    B_h2o = FT02wb['AP'+str(x)].value
                    B_h2o_list.append(B_h2o)
#this part creates the DataFrame after creating a list pair

A_h2o_list_pair = list(zip(A_h2o_list, A_h2o_datelist[0:]))
A_h2o_df = pd.DataFrame(A_h2o_list_pair)
A_h2o_df.columns = ['MS_A_moisture', 'DATETIME'] ##check if list title is updated
A_h2o_df.set_index('DATETIME', inplace=True)
A_h2o_df = A_h2o_df.loc[~A_h2o_df.index.duplicated(keep = 'last')]



B_h2o_list_pair = list(zip(B_h2o_list, B_h2o_datelist[0:]))
B_h2o_df = pd.DataFrame(B_h2o_list_pair)
B_h2o_df.columns = ['MS_B_moisture', 'DATETIME'] ##check if list title is updated
B_h2o_df.set_index('DATETIME', inplace=True)
B_h2o_df = B_h2o_df.loc[~B_h2o_df.index.duplicated(keep = 'last')]

#-----End for S code

#PS_10D
#PS_10Dde for creating a dataframe for MS_A analysis result for Element PS_9D
A_PS_10D_datelist = list()
A_PS_10D_datelist[:] = []
B_PS_10D_datelist = list()
B_PS_10D_datelist[:] = []

A_PS_10D_list = list()
A_PS_10D_list[:] = []
B_PS_10D_list = list()	
B_PS_10D_list[:] = []	

A_PS_10D_list_pair = list()
A_PS_10D_list_pair[:] = []
B_PS_10D_list_pair = list()
B_PS_10D_list_pair[:] = []

for y in range(0, len(Smpl_day_index)):   
    
    if Smpl_day_index[y] != Smpl_day_index[-1]:      
        for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
            if FT02wb['D'+str(x)].value != None:    
                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'A':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'A',(FT02wb['D'+str(x)].value))
                    A_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(parse(A_date))
                    A_PS_10D_datelist.append(parse(A_date))  
                    A_PS_10D = FT02wb['AQ'+str(x)].value
                    A_PS_10D_list.append(A_PS_10D)
                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'B':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'B',(FT02wb['D'+str(x)].value))
                    B_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(B_datelist)
                    B_PS_10D_datelist.append(parse(B_date)) 
                    B_PS_10D = FT02wb['AQ'+str(x)].value
                    B_PS_10D_list.append(B_PS_10D)
             
    if Smpl_day_index[y] == Smpl_day_index[-1]:
        for x in range(Smpl_day_index[y], smpl_end_index):
            if FT02wb['D'+str(x)].value != None:    
                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'A':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'A',(FT02wb['D'+str(x)].value))
                    A_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(A_date) 
                    A_PS_10D_datelist.append(parse(A_date))
                    A_PS_10D = FT02wb['AQ'+str(x)].value
                    A_PS_10D_list.append(A_PS_10D)

                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'B':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'B',(FT02wb['D'+str(x)].value))
                    B_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(B_date)
                    B_PS_10D_datelist.append(parse(B_date))
                    B_PS_10D = FT02wb['AQ'+str(x)].value
                    B_PS_10D_list.append(B_PS_10D)
#this part creates the DataFrame after creating a list pair

A_PS_10D_list_pair = list(zip(A_PS_10D_list, A_PS_10D_datelist[0:]))
A_PS_10D_df = pd.DataFrame(A_PS_10D_list_pair)
A_PS_10D_df.columns = ['MS_A_PS_10D', 'DATETIME'] ##check if list title is updated
A_PS_10D_df.set_index('DATETIME', inplace=True)
A_PS_10D_df = A_PS_10D_df.loc[~A_PS_10D_df.index.duplicated(keep = 'last')]



B_PS_10D_list_pair = list(zip(B_PS_10D_list, B_PS_10D_datelist[0:]))
B_PS_10D_df = pd.DataFrame(B_PS_10D_list_pair)
B_PS_10D_df.columns = ['MS_B_PS_10D', 'DATETIME'] ##check if list title is updated
B_PS_10D_df.set_index('DATETIME', inplace=True)
B_PS_10D_df = B_PS_10D_df.loc[~B_PS_10D_df.index.duplicated(keep = 'last')]

#-----End for PS_9D code

#PS_50D
#PS_50Dde for creating a dataframe for MS_A analysis result for Element PS_50D
A_PS_50D_datelist = list()
A_PS_50D_datelist[:] = []
B_PS_50D_datelist = list()
B_PS_50D_datelist[:] = []

A_PS_50D_list = list()
A_PS_50D_list[:] = []
B_PS_50D_list = list()	
B_PS_50D_list[:] = []	

A_PS_50D_list_pair = list()
A_PS_50D_list_pair[:] = []
B_PS_50D_list_pair = list()
B_PS_50D_list_pair[:] = []

for y in range(0, len(Smpl_day_index)):   
    
    if Smpl_day_index[y] != Smpl_day_index[-1]:      
        for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
            if FT02wb['D'+str(x)].value != None:    
                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'A':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'A',(FT02wb['D'+str(x)].value))
                    A_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(parse(A_date))
                    A_PS_50D_datelist.append(parse(A_date))  
                    A_PS_50D = FT02wb['AR'+str(x)].value
                    A_PS_50D_list.append(A_PS_50D)
                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'B':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'B',(FT02wb['D'+str(x)].value))
                    B_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(B_datelist)
                    B_PS_50D_datelist.append(parse(B_date)) 
                    B_PS_50D = FT02wb['AR'+str(x)].value
                    B_PS_50D_list.append(B_PS_50D)
             
    if Smpl_day_index[y] == Smpl_day_index[-1]:
        for x in range(Smpl_day_index[y], smpl_end_index):
            if FT02wb['D'+str(x)].value != None:    
                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'A':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'A',(FT02wb['D'+str(x)].value))
                    A_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(A_date) 
                    A_PS_50D_datelist.append(parse(A_date))
                    A_PS_50D = FT02wb['AR'+str(x)].value
                    A_PS_50D_list.append(A_PS_50D)

                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'B':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'B',(FT02wb['D'+str(x)].value))
                    B_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(B_date)
                    B_PS_50D_datelist.append(parse(B_date))
                    B_PS_50D = FT02wb['AR'+str(x)].value
                    B_PS_50D_list.append(B_PS_50D)
#this part creates the DataFrame after creating a list pair

A_PS_50D_list_pair = list(zip(A_PS_50D_list, A_PS_50D_datelist[0:]))
A_PS_50D_df = pd.DataFrame(A_PS_50D_list_pair)
A_PS_50D_df.columns = ['MS_A_PS_50D', 'DATETIME'] ##check if list title is updated
A_PS_50D_df.set_index('DATETIME', inplace=True)
A_PS_50D_df = A_PS_50D_df.loc[~A_PS_50D_df.index.duplicated(keep = 'last')]


B_PS_50D_list_pair = list(zip(B_PS_50D_list, B_PS_50D_datelist[0:]))
B_PS_50D_df = pd.DataFrame(B_PS_50D_list_pair)
B_PS_50D_df.columns = ['MS_B_PS_50D', 'DATETIME'] ##check if list title is updated
B_PS_50D_df.set_index('DATETIME', inplace=True)
B_PS_50D_df = B_PS_50D_df.loc[~B_PS_50D_df.index.duplicated(keep = 'last')]

#-----End for PS_50D code

#PS_90D
#PS_90Dde for creating a dataframe for MS_A analysis result for Element PS_90D
A_PS_90D_datelist = list()
A_PS_90D_datelist[:] = []
B_PS_90D_datelist = list()
B_PS_90D_datelist[:] = []

A_PS_90D_list = list()
A_PS_90D_list[:] = []
B_PS_90D_list = list()	
B_PS_90D_list[:] = []	

A_PS_90D_list_pair = list()
A_PS_90D_list_pair[:] = []
B_PS_90D_list_pair = list()
B_PS_90D_list_pair[:] = []

for y in range(0, len(Smpl_day_index)):   
    
    if Smpl_day_index[y] != Smpl_day_index[-1]:      
        for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
            if FT02wb['D'+str(x)].value != None:    
                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'A':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'A',(FT02wb['D'+str(x)].value))
                    A_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(parse(A_date))
                    A_PS_90D_datelist.append(parse(A_date))  
                    A_PS_90D = FT02wb['AS'+str(x)].value
                    A_PS_90D_list.append(A_PS_90D)
                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'B':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'B',(FT02wb['D'+str(x)].value))
                    B_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(B_datelist)
                    B_PS_90D_datelist.append(parse(B_date)) 
                    B_PS_90D = FT02wb['AS'+str(x)].value
                    B_PS_90D_list.append(B_PS_90D)
             
    if Smpl_day_index[y] == Smpl_day_index[-1]:
        for x in range(Smpl_day_index[y], smpl_end_index):
            if FT02wb['D'+str(x)].value != None:    
                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'A':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'A',(FT02wb['D'+str(x)].value))
                    A_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(A_date) 
                    A_PS_90D_datelist.append(parse(A_date))
                    A_PS_90D = FT02wb['AS'+str(x)].value
                    A_PS_90D_list.append(A_PS_90D)

                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'B':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'B',(FT02wb['D'+str(x)].value))
                    B_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(B_date)
                    B_PS_90D_datelist.append(parse(B_date))
                    B_PS_90D = FT02wb['AS'+str(x)].value
                    B_PS_90D_list.append(B_PS_90D)
#this part creates the DataFrame after creating a list pair

A_PS_90D_list_pair = list(zip(A_PS_90D_list, A_PS_90D_datelist[0:]))
A_PS_90D_df = pd.DataFrame(A_PS_90D_list_pair)
A_PS_90D_df.columns = ['MS_A_PS_90D', 'DATETIME'] ##check if list title is updated
A_PS_90D_df.set_index('DATETIME', inplace=True)
A_PS_90D_df = A_PS_90D_df.loc[~A_PS_90D_df.index.duplicated(keep = 'last')]


B_PS_90D_list_pair = list(zip(B_PS_90D_list, B_PS_90D_datelist[0:]))
B_PS_90D_df = pd.DataFrame(B_PS_90D_list_pair)
B_PS_90D_df.columns = ['MS_B_PS_90D', 'DATETIME'] ##check if list title is updated
B_PS_90D_df.set_index('DATETIME', inplace=True)
B_PS_90D_df = B_PS_90D_df.loc[~B_PS_90D_df.index.duplicated(keep = 'last')]

#-----End for PS_90D code

#PS_mean_v
#PS_mean_vde for creating a dataframe for MS_A analysis result for Element PS_mean_v
A_PS_mean_v_datelist = list()
A_PS_mean_v_datelist[:] = []
B_PS_mean_v_datelist = list()
B_PS_mean_v_datelist[:] = []

A_PS_mean_v_list = list()
A_PS_mean_v_list[:] = []
B_PS_mean_v_list = list()	
B_PS_mean_v_list[:] = []	

A_PS_mean_v_list_pair = list()
A_PS_mean_v_list_pair[:] = []
B_PS_mean_v_list_pair = list()
B_PS_mean_v_list_pair[:] = []

for y in range(0, len(Smpl_day_index)):   
    
    if Smpl_day_index[y] != Smpl_day_index[-1]:      
        for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
            if FT02wb['D'+str(x)].value != None:    
                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'A':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'A',(FT02wb['D'+str(x)].value))
                    A_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(parse(A_date))
                    A_PS_mean_v_datelist.append(parse(A_date))  
                    A_PS_mean_v = FT02wb['AT'+str(x)].value
                    A_PS_mean_v_list.append(A_PS_mean_v)
                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'B':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'B',(FT02wb['D'+str(x)].value))
                    B_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(B_datelist)
                    B_PS_mean_v_datelist.append(parse(B_date)) 
                    B_PS_mean_v = FT02wb['AT'+str(x)].value
                    B_PS_mean_v_list.append(B_PS_mean_v)
             
    if Smpl_day_index[y] == Smpl_day_index[-1]:
        for x in range(Smpl_day_index[y], smpl_end_index):
            if FT02wb['D'+str(x)].value != None:    
                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'A':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'A',(FT02wb['D'+str(x)].value))
                    A_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(A_date) 
                    A_PS_mean_v_datelist.append(parse(A_date))
                    A_PS_mean_v = FT02wb['AT'+str(x)].value
                    A_PS_mean_v_list.append(A_PS_mean_v)

                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'B':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'B',(FT02wb['D'+str(x)].value))
                    B_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(B_date)
                    B_PS_mean_v_datelist.append(parse(B_date))
                    B_PS_mean_v = FT02wb['AT'+str(x)].value
                    B_PS_mean_v_list.append(B_PS_mean_v)
#this part creates the DataFrame after creating a list pair

A_PS_mean_v_list_pair = list(zip(A_PS_mean_v_list, A_PS_mean_v_datelist[0:]))
A_PS_mean_v_df = pd.DataFrame(A_PS_mean_v_list_pair)
A_PS_mean_v_df.columns = ['MS_A_PS_mean_v', 'DATETIME'] ##check if list title is updated
A_PS_mean_v_df.set_index('DATETIME', inplace=True)
A_PS_mean_v_df = A_PS_mean_v_df.loc[~A_PS_mean_v_df.index.duplicated(keep = 'last')]



B_PS_mean_v_list_pair = list(zip(B_PS_mean_v_list, B_PS_mean_v_datelist[0:]))
B_PS_mean_v_df = pd.DataFrame(B_PS_mean_v_list_pair)
B_PS_mean_v_df.columns = ['MS_B_PS_mean_v', 'DATETIME'] ##check if list title is updated
B_PS_mean_v_df.set_index('DATETIME', inplace=True)
B_PS_mean_v_df = B_PS_mean_v_df.loc[~B_PS_mean_v_df.index.duplicated(keep = 'last')]


#-----End for PS_mean_v code

#PS_stdev
#PS_stdevde for creating a dataframe for MS_A analysis result for Element PS_stdev
A_PS_stdev_datelist = list()
A_PS_stdev_datelist[:] = []
B_PS_stdev_datelist = list()
B_PS_stdev_datelist[:] = []

A_PS_stdev_list = list()
A_PS_stdev_list[:] = []
B_PS_stdev_list = list()	
B_PS_stdev_list[:] = []

A_PS_stdev_list_pair = list()
A_PS_stdev_list_pair[:] = []
B_PS_stdev_list_pair = list()
B_PS_stdev_list_pair[:] = []

for y in range(0, len(Smpl_day_index)):   
    
    if Smpl_day_index[y] != Smpl_day_index[-1]:      
        for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
            if FT02wb['D'+str(x)].value != None:    
                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'A':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'A',(FT02wb['D'+str(x)].value))
                    A_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(parse(A_date))
                    A_PS_stdev_datelist.append(parse(A_date))  
                    A_PS_stdev = FT02wb['AU'+str(x)].value
                    A_PS_stdev_list.append(A_PS_stdev)
                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'B':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'B',(FT02wb['D'+str(x)].value))
                    B_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(B_datelist)
                    B_PS_stdev_datelist.append(parse(B_date)) 
                    B_PS_stdev = FT02wb['AU'+str(x)].value
                    B_PS_stdev_list.append(B_PS_stdev)
             
    if Smpl_day_index[y] == Smpl_day_index[-1]:
        for x in range(Smpl_day_index[y], smpl_end_index):
            if FT02wb['D'+str(x)].value != None:    
                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'A':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'A',(FT02wb['D'+str(x)].value))
                    A_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(A_date) 
                    A_PS_stdev_datelist.append(parse(A_date))
                    A_PS_stdev = FT02wb['AU'+str(x)].value
                    A_PS_stdev_list.append(A_PS_stdev)

                if filter_detrmin(str(FT02wb['C'+str(x)].value)) == 'B':
                    #print(FT02wb['B'+str(Smpl_day_index[y])].value, 'B',(FT02wb['D'+str(x)].value))
                    B_date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(FT02wb['B'+str(Smpl_day_index[y])].value))\
                    +' '+time_cleaner(str(FT02wb['D'+str(x)].value))
                    #print(B_date)
                    B_PS_stdev_datelist.append(parse(B_date))
                    B_PS_stdev = FT02wb['AU'+str(x)].value
                    B_PS_stdev_list.append(B_PS_stdev)
#this part creates the DataFrame after creating a list pair

A_PS_stdev_list_pair = list(zip(A_PS_stdev_list, A_PS_stdev_datelist[0:]))
A_PS_stdev_df = pd.DataFrame(A_PS_stdev_list_pair)
A_PS_stdev_df.columns = ['MS_A_PS_stdev', 'DATETIME'] ##check if list title is updated
A_PS_stdev_df.set_index('DATETIME', inplace=True)
A_PS_stdev_df = A_PS_stdev_df.loc[~A_PS_stdev_df.index.duplicated(keep = 'last')]

B_PS_stdev_list_pair = list(zip(B_PS_stdev_list, B_PS_stdev_datelist[0:]))
B_PS_stdev_df = pd.DataFrame(B_PS_stdev_list_pair)
B_PS_stdev_df.columns = ['MS_B_PS_stdev', 'DATETIME'] ##check if list title is updated
B_PS_stdev_df.set_index('DATETIME', inplace=True)
B_PS_stdev_df = B_PS_stdev_df.loc[~B_PS_stdev_df.index.duplicated(keep = 'last')]


#-----End for PS_stdev code



In [38]:
#Pressure Filter A lab results in merged dataframe
df_A_PbZn = A_Pb_df.join(A_Zn_df, how='outer')
df_A_CuNi = A_Cu_df.join(A_Ni_df, how='outer')
df_A_CoFe = A_Co_df.join(A_Fe_df, how='outer')
df_A_MnCr = A_Mn_df.join(A_Cr_df, how='outer')
df_A_CaSi = A_Ca_df.join(A_Si_df, how='outer')
df_A_AlMg = A_Al_df.join(A_Mg_df, how='outer')
df_A_Sh2o = A_S_df.join(A_h2o_df, how='outer')
df_A_PS_10D50D = A_PS_10D_df.join(A_PS_50D_df, how='outer')
df_A_PS_90Dmean = A_PS_90D_df.join(A_PS_mean_v_df, how='outer')

In [39]:
df_A_PbZnCuNi = df_A_PbZn.join(df_A_CuNi, how='outer')
df_A_CoFeMnCr = df_A_CoFe.join(df_A_MnCr, how='outer')
df_A_CaSiAlMg = df_A_CaSi.join(df_A_AlMg, how='outer')
df_A_Sh2oPS10D50D = df_A_Sh2o.join(df_A_PS_10D50D, how='outer')

In [40]:
df_A_PS_90Dmeanstdev = df_A_PS_90Dmean.join(A_PS_stdev_df, how='outer')

In [41]:
df_A_PbZnCuNiCoFeMnCr = df_A_PbZnCuNi.join(df_A_CoFeMnCr, how='outer')

In [42]:
df_A_CaSiAlMgSh2oPS10D50D = df_A_CaSiAlMg.join(df_A_Sh2oPS10D50D, how='outer')

In [43]:
df_A_CaSiAlMgSh2oPS10D50D90Dmeanstdev = df_A_CaSiAlMgSh2oPS10D50D.join(df_A_PS_90Dmeanstdev, how='outer')
df_A_PbZnCuNiCoFeMnCrCaSiAlMgSh2oPS10D50D90Dmeanstdev = df_A_PbZnCuNiCoFeMnCr.join(df_A_CaSiAlMgSh2oPS10D50D90Dmeanstdev, how='outer')
df_A_merged_lab_res = df_A_PbZnCuNiCoFeMnCrCaSiAlMgSh2oPS10D50D90Dmeanstdev

In [44]:
#Pressure Filter B lab results in merged DF
df_B_PbZn = B_Pb_df.join(B_Zn_df, how='outer')
df_B_CuNi = B_Cu_df.join(B_Ni_df, how='outer')
df_B_CoFe = B_Co_df.join(B_Fe_df, how='outer')
df_B_MnCr = B_Mn_df.join(B_Cr_df, how='outer')
df_B_CaSi = B_Ca_df.join(B_Si_df, how='outer')
df_B_AlMg = B_Al_df.join(B_Mg_df, how='outer')
df_B_Sh2o = B_S_df.join(B_h2o_df, how='outer')
df_B_PS_D50D = B_PS_10D_df.join(B_PS_50D_df, how='outer')
df_B_PS_90Dmean = B_PS_90D_df.join(B_PS_mean_v_df, how='outer')

df_B_PS_90Dmeanstdev = df_B_PS_90Dmean.join(B_PS_stdev_df, how='outer')
df_B_PbZnCuNi = df_B_PbZn.join(df_B_CuNi, how='outer')
df_B_CoFeMnCr = df_B_CoFe.join(df_B_MnCr, how='outer')
df_B_CaSiAlMg = df_B_CaSi.join(df_B_AlMg, how='outer')
df_B_Sh2o9D50D = df_B_Sh2o.join(df_B_PS_D50D, how='outer')

df_B_PbZnCuNiCoFeMnCr = df_B_PbZnCuNi.join(df_B_CoFeMnCr, how='outer')
df_B_CaSiAlMgSh2o9D50D = df_B_CaSiAlMg.join(df_B_Sh2o9D50D, how='outer')

df_B_PbZnCuNiCoFeMnCrCaSiAlMgSh2o9D50D = df_B_PbZnCuNiCoFeMnCr.join(df_B_CaSiAlMgSh2o9D50D, how='outer')
df_B_PbZnCuNiCoFeMnCrCaSiAlMgSh2o9D50D90Dmeanstdev = df_B_PbZnCuNiCoFeMnCrCaSiAlMgSh2o9D50D.join(df_B_PS_90Dmeanstdev, how='outer')

df_B_merged_lab_res = df_B_PbZnCuNiCoFeMnCrCaSiAlMgSh2o9D50D90Dmeanstdev

#deletes the rows whose index has duplicates but keeps the last value
df_A_merged_lab_res = df_A_merged_lab_res.loc[~df_A_merged_lab_res.index.duplicated(keep='last')]
df_B_merged_lab_res = df_B_merged_lab_res.loc[~df_B_merged_lab_res.index.duplicated(keep='last')]

print('Done', datetime.datetime.now())

Done 2020-01-27 14:24:31.101441


In [45]:
df_B_merged_lab_res

,MS_B_Pb,MS_B_Zn,MS_B_Cu,MS_B_Ni,MS_B_Co,MS_B_Fe,MS_B_Mn,MS_B_Cr,MS_B_Ca,MS_B_Si,MS_B_Al,MS_B_Mg,MS_B_S,MS_B_moisture,MS_B_PS_10D,MS_B_PS_50D,MS_B_PS_90D,MS_B_PS_mean_v,MS_B_PS_stdev
DATETIME,,,,,,,,,,,,,,,,,,,
2020-01-01 12:19:00,0.001,0.0134,0.001,57.5493,5.1232,0.7132,0.0127,0.0136,0.0008,0.01,0.0202,0.0130,35.1782,11.83,39.3980,79.3725,150.9813,88.67963,49.10050
2020-01-01 21:26:00,0.001,0.0224,0.001,57.7865,5.1674,0.7226,0.0073,0.0139,0.0014,0.01,0.0111,0.0095,35.2813,12.11,42.0286,79.5373,148.4398,88.79488,47.50780
2020-01-01 05:50:00,0.001,0.0241,0.001,57.6686,5.2138,0.7917,0.0083,0.0146,0.0013,0.01,0.0152,0.0120,35.3354,11.48,47.1176,82.5196,82.5294,152.29320,92.62222
2020-01-02 10:30:00,0.001,0.0228,0.001,57.8486,5.2758,0.7733,0.0107,0.0131,0.0013,0.01,0.0197,0.0131,35.1331,11.42,47.1728,84.4705,154.9832,94.14171,47.93490
2020-01-02 21:40:00,0.001,0.0183,0.001,57.9900,5.3082,0.6622,0.0063,0.0120,0.0013,0.01,0.0102,0.0089,35.0078,12.18,41.9633,82.9174,159.8658,93.19717,50.67190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-25 14:00:00,0.001,0.0139,0.001,57.9773,5.3652,0.5429,0.0045,0.0108,0.0013,0.01,0.0059,0.0055,34.8467,13.56,40.3411,77.0120,142.4377,85.46300,43.99740
2020-01-25 21:32:00,0.001,0.0158,0.001,57.7833,5.1965,0.5025,0.0083,0.0107,0.0010,0.01,0.0116,0.0114,35.4042,12.69,41.3524,76.9939,140.0300,85.17178,42.64070
2020-01-25 03:06:00,0.001,0.0110,0.001,57.8854,5.3229,0.4737,0.0050,0.0101,0.0021,0.01,0.0055,0.0078,35.2874,14.16,40.7107,77.2816,139.2003,84.85207,42.67520


# COMBINE PiDATA WITH LABORATORY RESULTS FOR ALPHA

In [46]:
#fcn for formatting the dates from range str
def datesplitter_start(x):
    return parse(x.split('to')[0])
def datesplitter_end(x):
    return parse(x.split('to')[1])
#---------
df_PressureFilter_Param['newdt'] = df_PressureFilter_Param.index
df_PressureFilter_Param['startdate'] = df_PressureFilter_Param['newdt'].apply(lambda x: datesplitter_start(x))
df_PressureFilter_Param['enddate'] = df_PressureFilter_Param['newdt'].apply(lambda x: datesplitter_end(x))

#code for changing the index to Datetime in range format (from date1 to date2)
df_anl_indexList = list()
df_anl_indexList[:] = []
df_n11_indexList = list()
df_n11_indexList[:] = []
try:
    for i in range(0, len(df_A_merged_lab_res)):
        for j in range(0, len(dfn11)):
            if dfn11['enddate'][j-1] <= df_A_merged_lab_res.index[i] and dfn11['enddate'][j] >= df_A_merged_lab_res.index[i]:
                df_n11_indexList.append(dfn11.index[j])
                df_anl_indexList.append(df_A_merged_lab_res.index[i]) 
except IndexError:
    print('IndexError')  
Analys_indexList_Pair = list(zip(df_anl_indexList, df_n11_indexList[0:]))
dfEL4 = pd.DataFrame(Analys_indexList_Pair)
dfEL4.columns = ['DATETIME', 'FT_cycletimerange']
dfEL4.set_index('DATETIME', inplace=True)

dfEL5 = df_A_merged_lab_res.join(dfEL4, how='outer')
dfEL5.columns = ['MS_A_Pb_spcl', 'MS_A_Zn_spcl', 'MS_A_Cu_spcl', 'MS_A_Ni_spcl', 'MS_A_Co_spcl', 'MS_A_Fe_spcl',
       'MS_A_Mn_spcl', 'MS_A_Cr_spcl', 'MS_A_Ca_spcl', 'MS_A_Si_spcl', 'MS_A_Al_spcl', 'MS_A_Mg_spcl','MS_A_S_spcl', 'MS_A_moisture_spcl', 'MS_A_PS_10D_spcl', 'MS_A_PS_50D_spcl', 'MS_A_PS_90D_spcl',
       'MS_A_PS_mean_v_spcl', 'MS_A_PS_stdev_spcl','DATETIME']


dfEL5.set_index('DATETIME', inplace=True)
df_Cycle_LabRes = dfn11.join(dfEL5, how='outer')
df_ffillna = df_Cycle_LabRes.fillna(method='ffill')
df_Cycle_LabRes['Cycle_Count'] = np.arange(1, len(df_Cycle_LabRes['A_FILT_T'])+1)
df_Cycle_LabRes_2=df_Cycle_LabRes.reindex(columns= ['Cycle_Count','A_PU26A_CUR', 'A_PU27A_CUR', 'A_N2DRY', 'A_CWP_1', 'A_CWP_2',
       'A_FILT_T', 'A_MANF_P', 'A_CWSH1', 'A_CWSH2', 'A_PRESS1_t',
       'A_PRESS2_t', 'A_PU26A_OP', 'A_PU27A_OP', 'newdt', 'startdate',
       'enddate', 'MS_A_Pb_spcl', 'MS_A_Zn_spcl', 'MS_A_Cu_spcl',
       'MS_A_Ni_spcl', 'MS_A_Co_spcl', 'MS_A_Fe_spcl', 'MS_A_Mn_spcl',
       'MS_A_Cr_spcl', 'MS_A_Ca_spcl', 'MS_A_Si_spcl', 'MS_A_Al_spcl',
       'MS_A_Mg_spcl', 'MS_A_S_spcl', 'MS_A_moisture_spcl', 'MS_A_PS_10D_spcl',
       'MS_A_PS_50D_spcl', 'MS_A_PS_90D_spcl', 'MS_A_PS_mean_v_spcl',
       'MS_A_PS_stdev_spcl'])
df_Cycle_LabRes_2_ffillna = df_Cycle_LabRes_2.fillna(method='ffill')
df_JAN2020_cycLab_A= df_Cycle_LabRes_2_ffillna

#---------------dfEL5 is the is the analysis results data indexed by daterange format. readt for merging with dfn11(FT02A data from Pi)

# COMBINE PiDATA WITH LABORATORY RESULTS FOR BRAVO

In [47]:
#fcn for formatting the dates from range str
def datesplitter_start(x):
    return parse(x.split('to')[0])
def datesplitter_end(x):
    return parse(x.split('to')[1])
#---------
df_PressureFilter_Param_B['newdt'] = df_PressureFilter_Param_B.index
df_PressureFilter_Param_B['startdate'] = df_PressureFilter_Param_B['newdt'].apply(lambda x: datesplitter_start(x))
df_PressureFilter_Param_B['enddate'] = df_PressureFilter_Param_B['newdt'].apply(lambda x: datesplitter_end(x))

#code for changing the index to Datetime in range format (from date1 to date2)
df_anlB_indexList = list()
df_anlB_indexList[:] = []
df_n11B_indexList = list()
df_n11B_indexList[:] = []
try:
    for i in range(0, len(df_B_merged_lab_res)):
        for j in range(0, len(dfn11B)):
            if dfn11B['enddate'][j-1] <= df_B_merged_lab_res.index[i] and dfn11B['enddate'][j] >= df_B_merged_lab_res.index[i]:
                df_n11B_indexList.append(dfn11B.index[j])
                df_anlB_indexList.append(df_B_merged_lab_res.index[i]) 
except IndexError:
    print('IndexError')  
Analys_indexList_Pair = list(zip(df_anlB_indexList, df_n11B_indexList[0:]))
dfEL4B = pd.DataFrame(Analys_indexList_Pair)
dfEL4B.columns = ['DATETIME', 'FT_cycletimerange']
dfEL4B.set_index('DATETIME', inplace=True)

dfEL5B = df_B_merged_lab_res.join(dfEL4B, how='outer')
dfEL5B.columns = ['MS_B_Pb_spcl', 'MS_B_Zn_spcl', 'MS_B_Cu_spcl', 'MS_B_Ni_spcl', 'MS_B_Co_spcl', 'MS_B_Fe_spcl',
       'MS_B_Mn_spcl', 'MS_B_Cr_spcl', 'MS_B_Ca_spcl', 'MS_B_Si_spcl', 'MS_B_Al_spcl', 'MS_B_Mg_spcl','MS_B_S_spcl', 'MS_B_moisture_spcl', 'MS_B_PS_10D_spcl', 'MS_B_PS_50D_spcl', 'MS_B_PS_90D_spcl',
       'MS_B_PS_mean_v_spcl', 'MS_B_PS_stdev_spcl','DATETIME']

dfEL5B.set_index('DATETIME', inplace=True)
df_Cycle_LabResB = dfn11B.join(dfEL5B, how='outer')
df_ffillnaB = df_Cycle_LabResB.fillna(method='ffill')
df_Cycle_LabResB['Cycle_Count'] = np.arange(1, len(df_Cycle_LabResB['B_FILT_T'])+1)
df_Cycle_LabRes_2B=df_Cycle_LabResB.reindex(columns= ['Cycle_Count','B_PU26B_CUR', 'B_PU27B_CUR', 'B_N2DRY', 'B_CWP_1', 'B_CWP_2',
       'B_FILT_T', 'B_MANF_P', 'B_CWSH1', 'B_CWSH2', 'B_PRESS1_t',
       'B_PRESS2_t', 'B_PU26B_OP', 'B_PU27B_OP', 'newdt', 'startdate',
       'enddate', 'MS_B_Pb_spcl', 'MS_B_Zn_spcl', 'MS_B_Cu_spcl',
       'MS_B_Ni_spcl', 'MS_B_Co_spcl', 'MS_B_Fe_spcl', 'MS_B_Mn_spcl',
       'MS_B_Cr_spcl', 'MS_B_Ca_spcl', 'MS_B_Si_spcl', 'MS_B_Al_spcl',
       'MS_B_Mg_spcl', 'MS_B_S_spcl', 'MS_B_moisture_spcl', 'MS_B_PS_10D_spcl',
       'MS_B_PS_50D_spcl', 'MS_B_PS_90D_spcl', 'MS_B_PS_mean_v_spcl',
       'MS_B_PS_stdev_spcl'])
df_Cycle_LabRes_2B_ffillna = df_Cycle_LabRes_2B.fillna(method='ffill')
df_JAN2020_cycLab_B = df_Cycle_LabRes_2B_ffillna

#---------------dfEL5 is the is the analysis results data indexed by daterange format. readt for merging with dfn11(FT02A data from Pi)

# SAVE THE DATAFRAME TO EXCEL

In [48]:
df_JAN2020_cycLab_A.to_excel(r'C:\Users\v.t.flores\Documents\106FT02A_data_JAN2020.xlsx')
df_JAN2020_cycLab_A.to_excel(r'\\THGOBNA002\thpal\05 Production\150 DCS\00 DCS Monitoring & Records\00 Active Files\2020\Pressure_Filter_Data_2019\106FT02A_data_JAN2020.xlsx')

In [49]:
df_JAN2020_cycLab_B.to_excel(r'C:\Users\v.t.flores\Documents\106FT02B_data_JAN2020.xlsx')
df_JAN2020_cycLab_B.to_excel(r'\\THGOBNA002\thpal\05 Production\150 DCS\00 DCS Monitoring & Records\00 Active Files\2020\Pressure_Filter_Data_2019\106FT02B_data_JAN2020.xlsx')